In [2]:
import csv
from konlpy.tag import Kkma
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

kkma = Kkma()

f = open("불용어 리스트.txt",'r', encoding='utf8')
stop_words = f.read()
stop_words = stop_words.split('\n')

In [3]:
def split_sentence(doc):
    text = doc[2]
    sentence = kkma.sentences(text)
    return sentence

def split_morphs(doc):
    doc_morphs = kkma.morphs(doc)
    return doc_morphs

In [4]:
def preprocessing(text):
    text = text.replace('ㅋ','').replace('ㅠ','').replace('ㅜ','') # 'ㅋ', 'ㅠ', 'ㅜ' 제거
    
    text_pos = kkma.pos(text)
    text_morphs = kkma.morphs(text)
    
    words = []
    tags = ['NNG','NNP','NNB','NNM','NP','VV','VA','VXV','VXA','MDT','MDN','MAG','MAC','UN','XR']
    
    # 제거할 품사 tag에 해당하는 단어 리스트 만들기
    for word, tag in text_pos:
        if tag in tags:
            words.append(word)
            
    # 불용어 지우기
    result = []
    for w in words: 
        if w not in stop_words: 
            result.append(w)
            
    return result

# 영화 겟아웃

In [4]:
file_name = "겟아웃_최종본.csv"
uptitle= "겟아웃_상위tf.csv"
ctitle = "겟아웃_네이버평점.csv" 

csv_file = open(file_name, 'r')
doc = csv.reader(csv_file)
next(doc) # header 넘어가기
    
# 문장별
sentences = []
for line in doc:
    sentences.extend(split_sentence(line))
# 형태소별    
sentence_morphs = []
for s in sentences:
        sentence_morphs.append(preprocessing(s))    

for a in sentence_morphs:
    if a == []:
        sentence_morphs.remove(a)    

# 상위tf 2차 전처리   
f=open(uptitle,'r')
rf=csv.reader(f)
next(rf)
TF=[]
for row in rf:
    TF.append(row[0])
stop=['이', '을', '스', '여','노', '은', '아', '터', '드', '마', '자', '고', '레','거','개방','관람','씬','내용','결국','그런데',
      '거','늘', '의', '라', '누', '어', '건','진','자', '마','로','스','까','은','시','드','곽','왜','그리하','약간','갑자기',
      '마','저','거','라','까','분','늘','자','의','어','스','여','을','몇','몇몇','걸','시','드','단','서','인','블록버스터',
      '늘','메','파','자','김','티','리','바','스','건','대','새','어','보','시','디','르','을','고','예고편','스포일러','장르',
       '어쩌','듯하','경','어느','결말','이때','재미있','그리하','이번','조금','스','자','누','터','라','뜨','다','막','분','임',
      '위','하다','리','사','유','면','감독','스포','줄거리','영화']
newTF=[]
for word in TF:
    if word not in stop:
        newTF.append(word)
    newTF=[]
    for word in TF:
        if word not in stop:
            newTF.append(word)   

dataset2=[]
for i in range(len(sentences)):
    a=split_morphs(sentences[i])
    dataset= []
    for w in a:
        if w in newTF: 
            dataset.append(w)
    dataset2.append(dataset)
    
dataset3=[]
for k in range(len(dataset2)):
    if dataset2[k] != []:
        dataset3.append(dataset2[k])
 ############################ Apriori 알고리즘 적용 ################################
te = TransactionEncoder()
te_ary = te.fit(dataset3).transform(dataset3)
df = pd.DataFrame(te_ary, columns=te.columns_) #위에서 나온걸 보기 좋게 데이터프레임으로 변경
    
frequent_itemsets3 = apriori(df, min_support=0.003, max_len=3, use_colnames=True)
frequent_itemsets2 = apriori(df, min_support=0.003, max_len=2, use_colnames=True)

frequentThree=frequent_itemsets3[len(frequent_itemsets2):len(frequent_itemsets3)]
frequentThree=frequentThree.sort_values("support",ascending=False)

# 일정 확률 이상의 데이터 선택
frequentThree=frequentThree[frequentThree["support"]>=0.007]

# frozenset를 list로 변화
sets=[]
for line in frequentThree['itemsets']:
    sets.append(line)
association=[list(x) for x in sets]

In [5]:
############################### 나이브 베이지안 알고리즘 적용 ################################
spo = []
nonspo = []

for row in sentence_morphs:
    s1 = set(row)
    for dataset in association:
        s2 = set(dataset)
        isSpo = False
        if s2.issubset(s1) == True:
            spo.append(row)
            isSpo = True
            break
    if not isSpo:
        nonspo.append(row)
        
csv_file = open(ctitle, 'r')
doc = csv.reader(csv_file)
next(doc) # header 넘어가기

review_token = []
totalreview = []
for line in doc:
    text = line[2]
    totalreview.append(text) # totalreview 리스트에 결과 저장
    
index = 0
for comment in totalreview[0:1000]:
    review=preprocessing(comment) # 형태소 분석
    index+=1
    
    
    #__________prior probabilities
    prior_spo=len(spo)/(len(spo)+len(nonspo))
    prior_nonspo=len(nonspo)/(len(spo)+len(nonspo))
    
    #_________training data
    
    #spo의 중복단어 리스트 (the sum of term frequency for class SPO)
    spo_overlap=[]
    for i in range(len(spo)):
        spo_overlap.extend(spo[i])
        
    #nonspo의 중복단어 리스트 (the sum of term frequency for class NONSPO)
    nonspo_overlap=[]   
    for i in range(len(nonspo)):
        nonspo_overlap.extend(nonspo[i])
    
    #중복없는 전체단어 갯수: Vocabulary
    all_spo=[]
    for i in range(len(spo)):
        for j in range(len(spo[i])):
            if(spo[i][j] not in all_spo):
                all_spo.append(spo[i][j])
                
    all_nonspo=[]
    for i in range(len(nonspo)):
        for j in range(len(nonspo[i])):
            if(nonspo[i][j] not in all_nonspo):
                all_nonspo.append(nonspo[i][j])
    
    overall=[]
    overall.extend(all_spo)
    overall.extend(all_nonspo)
    
    nonover=[]
    for i in range(len(overall)):
        if(overall[i] not in nonover):
            nonover.append(overall[i])
    all_count=len(nonover)
    
    #_________likelihood
    
    #spo의  likelihood
    spo_like_result=1
    for i in range(len(review)):
        count=1# Laplace smoothing
        for j in range(len(spo_overlap)):
            if (review[i]==spo_overlap[j]):
                count=count+1
        spo_like_result=spo_like_result*(count/(len(spo_overlap)+all_count))#review 단어별 like
        
    
    #nonspo의  likelihood
    nonspo_like_result=1
    for i in range(len(review)):
        ncount=1# Laplace smoothing
        for j in range(len(nonspo_overlap)):
            if (review[i]==nonspo_overlap[j]):
                ncount=ncount+1
        nonspo_like_result=nonspo_like_result*(ncount/(len(nonspo_overlap)+all_count))#review 단어별 like 
        
    #_________posterior probabilities
    
    #P(SPO | REVIEW)=p(REVIEW|SPO)P(SPO)
    post_spo=spo_like_result*prior_spo
    
    #P(NONSPO | REVIEW)=P(REVIEW|NONSPO)P(NONSPO)
    post_nonspo=nonspo_like_result*prior_nonspo
    if(post_spo>post_nonspo):
        print(index,"번째 평점리뷰:",comment)
        print("분류결과: 스포 O")
        print("다음 review는 spo:{} >nonspo:{}이므로 스포이다.".format(post_spo,post_nonspo))
        print("=============================================================================================")
    
    elif(post_spo<post_nonspo):
        print(index,"번째 평점리뷰:",comment)
        print("분류결과: 스포 X")
        print("다음 review는 spo:{} <nonspo:{}이므로 스포가 아니다.".format(post_spo,post_nonspo))
        print("=============================================================================================")

        

1 번째 평점리뷰: 크리스가 2층으로 올라갈때 떠들썩하던 1층사람들 갑자기 조용해질때 소름 이장면이 제일 무서웠음  
분류결과: 스포 X
다음 review는 spo:3.9853584167846934e-40 <nonspo:9.061785575411402e-37이므로 스포가 아니다.
2 번째 평점리뷰: 마지막에 경찰차가 오자 자연스럽게 두손을 드는 주인공의 모습이 슬펐음 차별을 당연시 여기는 느낌이라..  
분류결과: 스포 X
다음 review는 spo:1.348131837155569e-36 <nonspo:7.977327211783189e-35이므로 스포가 아니다.
3 번째 평점리뷰: 와, 초반에 여친이 흑인남친 면허증 안보여 주려고 했던거 설계였네 경찰한테 꼬리 잡힐까봐,, 백인경찰이 편견으로 흑인 검문 때리는 줄 알았는데 지리네 설계  
분류결과: 스포 O
다음 review는 spo:1.6822020609495262e-61 >nonspo:1.34140653261866e-63이므로 스포이다.
4 번째 평점리뷰: 경찰차가 왔을때 모두 다 긴장했을듯 ㅋ  
분류결과: 스포 X
다음 review는 spo:1.7801165850941815e-17 <nonspo:5.242836834705659e-16이므로 스포가 아니다.
5 번째 평점리뷰: 근데 여자애는 그렇게 중요한 사진을 그리 대충 보관하냐  
분류결과: 스포 X
다음 review는 spo:1.3809756539093937e-29 <nonspo:4.755183042043415e-28이므로 스포가 아니다.
6 번째 평점리뷰: 다시는 커피 못저어먹는 영화  
분류결과: 스포 X
다음 review는 spo:1.1696278844368824e-17 <nonspo:1.8672580122109202e-16이므로 스포가 아니다.
7 번째 평점리뷰: 영화에서 그인간들 ㅋ. 왜그렇게 주인공이 담배를 못피게 하는지 나중에 생각해 보면 참 ㅋ 무섭다 ㅋ  
분류결과: 스포 X
다음 review는 spo:6.000063575569464e

35 번째 평점리뷰: 한시간 반짜리 영화인데 무슨 30분 영화인줄 엄청 몰입감 최고였다.  
분류결과: 스포 X
다음 review는 spo:6.917611777345883e-38 <nonspo:1.1351575464241991e-35이므로 스포가 아니다.
36 번째 평점리뷰: 여자하녀 웃으면서 눈물 흘리는 연기가 제일 인상적이였다. 빙고 할 때 처음에 뭐하는건가 싶었는데 앞에 사진 걸려 있는거 보고 소름  
분류결과: 스포 O
다음 review는 spo:4.135889981998784e-45 >nonspo:3.81550569691663e-47이므로 스포이다.
37 번째 평점리뷰: 관람객남자주인공 매력있음 빨간눈이 터질듯  
분류결과: 스포 O
다음 review는 spo:5.3980107236371676e-31 >nonspo:2.405429057459333e-31이므로 스포이다.
38 번째 평점리뷰: 흔하지않은 소재와 괜찮은 연출임 간만에 재밌게봄  
분류결과: 스포 X
다음 review는 spo:5.8146343322500776e-30 <nonspo:9.307878894109705e-28이므로 스포가 아니다.
39 번째 평점리뷰: 예상을 어느정도 해도 첨부터 쌓아올린 긴장감에 인물관계도 에 결말까지는 절대 집중해서 보지않고는 제대로 봤다고 할수 없는 영화  
분류결과: 스포 X
다음 review는 spo:6.306337052615771e-54 <nonspo:1.1791091463071784e-49이므로 스포가 아니다.
40 번째 평점리뷰: 여기다가써봐야 누가보겠냐만은 베스트댓글?에 여자애는 중요한사진을 대충보관하냐~써있길래 씁니다. 수술받기로한 짐허드슨이 영상통화할때 말해줍니다. 1,2,3단계가있는데 2단계가 "이 상황에대한인지"라고, 영화 한번 더 보시면 보입니다. 일부러 보라고놔둔거  
분류결과: 스포 X
다음 review는 spo:1.1272498105757908e-96 <nonspo:2.33122352626532e-92이므로 스포가 아니다.
41 번째 

68 번째 평점리뷰: 곡성'보다 끝이 명확하고 사이다스럽고, 어지럽지도 않아서 더 좋았습니다.  
분류결과: 스포 X
다음 review는 spo:8.558252812952796e-20 <nonspo:1.5130842235802774e-18이므로 스포가 아니다.
69 번째 평점리뷰: 초중반까진 서늘한 느낌에 오싹한 분위기도 있고, 이야기 흐름도 좋았는데 후반부에 너무 갑자기 끝나버림.  
분류결과: 스포 X
다음 review는 spo:4.3660806494135543e-42 <nonspo:1.5529351169593482e-40이므로 스포가 아니다.
70 번째 평점리뷰: 재미없단 사람치고 영화볼줄 아는사람 없도다  
분류결과: 스포 X
다음 review는 spo:1.4611798868032766e-10 <nonspo:2.563966279194885e-09이므로 스포가 아니다.
71 번째 평점리뷰: 남주 올라갈때 모두 멈추는 장면, 빙고 낙찰 장면 너무 기억에 남는다.. 모든 장면에 이유가 있고 그 모든게 떡밥이었음  
분류결과: 스포 X
다음 review는 spo:4.072320082173433e-52 <nonspo:1.2023289472886092e-48이므로 스포가 아니다.
72 번째 평점리뷰: 너무 충격적이여서 여운이 1주일은 갈거 같다  
분류결과: 스포 X
다음 review는 spo:1.2057070694438233e-24 <nonspo:4.628117921337504e-23이므로 스포가 아니다.
73 번째 평점리뷰: 평점이 왜 이러는지 간만에 정말 재밌게본 공포영화예요 꼭 보세요 후회안해요~  
분류결과: 스포 X
다음 review는 spo:1.3778982808534487e-39 <nonspo:1.5024288005532502e-35이므로 스포가 아니다.
74 번째 평점리뷰: 흑인 소울 없는 흑인이 얼마나 무서운지 알았다  
분류결과: 스포 X
다음 review는 spo:3.607710952380012e-18 <nonspo:5.628395429795

102 번째 평점리뷰: 우리나라 관객들은 예고편을 보고 "봐야겠다!"하고 봐서 그런가 영화에 '더' 기대했던 무언가가 들어있지 않다고 느껴졌다. 예고편이 절반 이상을 차지한 영화. 그럼에도 불구하고 감독이 던진 떡밥과 연출력은 곱씹을 수 있는 맛이 있었다.  
분류결과: 스포 X
다음 review는 spo:4.478379957979905e-80 <nonspo:5.646307396113001e-75이므로 스포가 아니다.
103 번째 평점리뷰: 첫 씬부터 도처에 모든걸 암시하는 장치들, 와 진짜 천재다  
분류결과: 스포 X
다음 review는 spo:9.691057220416798e-31 <nonspo:1.5324320350335218e-27이므로 스포가 아니다.
104 번째 평점리뷰: 예고편조차 보지 않은 나는 정말정말 신선했고 소름끼쳤다.  
분류결과: 스포 X
다음 review는 spo:2.768407332336313e-21 <nonspo:5.160255873412691e-18이므로 스포가 아니다.
105 번째 평점리뷰: 이제 흑형이라는 표현도 인종차별적 표현이 될 수 있다는 것을 이 '사이코'스러운 영화로 느낄 수 있었다. 과한 우러러봄도 차별적 요소라는 것.  
분류결과: 스포 X
다음 review는 spo:4.1395246866016534e-50 <nonspo:4.565562372378874e-48이므로 스포가 아니다.
106 번째 평점리뷰: 여자의 마음을 잡는거 처럼 이영화도 스포로 다 알고 봐도 깊이는 보지않고는 모르는법  
분류결과: 스포 X
다음 review는 spo:8.158578038628219e-27 <nonspo:2.7523593845227536e-25이므로 스포가 아니다.
107 번째 평점리뷰: 이 영화를 욕하는 사람들은 이 영화를 보지 않은 사람들이다  
분류결과: 스포 X
다음 review는 spo:1.1981675071786868e-09 <nonspo:4.1878115893516454e-08이므로 스포가 아니다.
108 번째 평점

135 번째 평점리뷰: 여자친구네 흑인 하인들이 왜그랫는지 이제 이해가된다.. 연출대단하다 정말  
분류결과: 스포 O
다음 review는 spo:3.362448206764013e-35 >nonspo:2.8905640346778586e-36이므로 스포이다.
136 번째 평점리뷰: 여러분 이래서 담배를 피면 안됩니다  
분류결과: 스포 X
다음 review는 spo:6.589854665973654e-18 <nonspo:1.1600312380782127e-17이므로 스포가 아니다.
137 번째 평점리뷰: 남편과 영후 본 이후로 제가 말할때마다 남편이 자꾸 커피잔을 저어요..묘학게 기분나빠요...  
분류결과: 스포 X
다음 review는 spo:2.2833400710335576e-50 <nonspo:7.906542417472455e-50이므로 스포가 아니다.
138 번째 평점리뷰: 말 그대로 신선한 영화 간만에 생각과 추리하게 되는 영화 잘 봤습니다 ^^  
분류결과: 스포 X
다음 review는 spo:1.356761727596893e-20 <nonspo:1.3179150557320014e-17이므로 스포가 아니다.
139 번째 평점리뷰: 관람객정말끝까지 보면 잼있어요.^^  
분류결과: 스포 X
다음 review는 spo:2.1174401197167164e-15 <nonspo:1.4088411265990604e-14이므로 스포가 아니다.
140 번째 평점리뷰: 진짜 무서워죽는줄 알았어요 컨저링같이 귀신나오는거나 파이널데스티네이션같이 잔인한건 꽤 잘보는편인데 이건 진짜 무서웠어요 역시 사람이 젤 무섭네요....  
분류결과: 스포 X
다음 review는 spo:3.6564162538788994e-74 <nonspo:4.0400941550122975e-69이므로 스포가 아니다.
141 번째 평점리뷰: 차 마실땐 종이컵으로...  
분류결과: 스포 X
다음 review는 spo:1.1404330823830451e-11 <nonspo:1.744194747751623e

...

850 번째 평점리뷰: 신선하다는 평이 가장 어울릴만한 영화에요!  
분류결과: 스포 X
다음 review는 spo:5.617498846272358e-25 <nonspo:1.7786492403571584e-22이므로 스포가 아니다.
851 번째 평점리뷰: 신선하고 질질끄는것 없이 깔끔했어요. 최근본 영화중 최고였어요  
분류결과: 스포 X
다음 review는 spo:3.973333460370886e-29 <nonspo:4.752896896350124e-27이므로 스포가 아니다.
852 번째 평점리뷰: 참신한 공포영화 소재도 참신하고 배우들이 연기를 잘해서 재밌게 관람했어요  
분류결과: 스포 X
다음 review는 spo:3.180526676686703e-36 <nonspo:1.2244506438922598e-30이므로 스포가 아니다.
853 번째 평점리뷰: 지루할 틈 없이 이야기가 진행되서 좋았습니다.  
분류결과: 스포 X
다음 review는 spo:5.1349516877716775e-18 <nonspo:1.479460129722938e-17이므로 스포가 아니다.
854 번째 평점리뷰: 장르의 모호함이 주는 신선함? 우정은 위대하다.  
분류결과: 스포 X
다음 review는 spo:2.0177896008282164e-23 <nonspo:2.401448716712831e-21이므로 스포가 아니다.
855 번째 평점리뷰: 난 두번봤다 꿀잼이다.  
분류결과: 스포 X
다음 review는 spo:7.127706764894032e-13 <nonspo:8.720973738758115e-12이므로 스포가 아니다.
856 번째 평점리뷰: 그냥 미친영화다..  심장 안좋은 분들은 보지마요.. 근데 제대로 느끼려면 영화관가서 보는게 맞음.. 감독은 백인에게 호되게 당했던적이 있는듯..  
분류결과: 스포 X
다음 review는 spo:1.1137377994829293e-56 <nonspo:1.538046680176425e-52이므로 스포가 아니다.
857 번째 평점리뷰:

886 번째 평점리뷰: 사람들의 행동을 통해 암시하는 스릴러,긴장감정말 좋은 영화다  
분류결과: 스포 X
다음 review는 spo:9.26568984700317e-21 <nonspo:8.144513322731563e-18이므로 스포가 아니다.
887 번째 평점리뷰: 좋았어요. 많은 생각을 하게되네요  
분류결과: 스포 X
다음 review는 spo:0.2517814726840855 <nonspo:0.7482185273159145이므로 스포가 아니다.
888 번째 평점리뷰: 우와 진짜 최근에 본영화중손에 꼽는다진짜강추!!  
분류결과: 스포 X
다음 review는 spo:2.980000095278165e-29 <nonspo:1.9363654022167178e-27이므로 스포가 아니다.
889 번째 평점리뷰: 시사회 보고 충격적이어서 한번 더 봤는데, 두번 보니까 더 이해되면서 더 충격 ㅋㅋ  
분류결과: 스포 X
다음 review는 spo:1.937078016795088e-22 <nonspo:1.852546152892755e-20이므로 스포가 아니다.
890 번째 평점리뷰: 어떻게 될지 알면서도 당한 느낌이야  
분류결과: 스포 X
다음 review는 spo:1.0077151824207182e-07 <nonspo:3.077108373983413e-07이므로 스포가 아니다.
891 번째 평점리뷰: 재치와 긴장감 그리고 통쾌함까지 갖춘 것이 많다.  
분류결과: 스포 X
다음 review는 spo:2.0166096378254441e-16 <nonspo:7.414953297889791e-16이므로 스포가 아니다.
892 번째 평점리뷰: 현실에서도 일어날법한 무서운 상상  
분류결과: 스포 X
다음 review는 spo:1.7116505625905593e-19 <nonspo:8.826324637551615e-18이므로 스포가 아니다.
893 번째 평점리뷰: 관람객스트레스받지않는 스릴러영화  
분류결과: 스포 X
다음 review는 spo:8.2680995150843

922 번째 평점리뷰: 관람객영하를 보면서 처음느껴보는 기분이였고요 신선하고 전반적으로 다 재미있었습니다 !! 근데 마지막이 너무 빨리 끝내려는 거 같아서 아쉬웠어요  
분류결과: 스포 X
다음 review는 spo:3.4312626090183556e-51 <nonspo:5.58392402858328e-48이므로 스포가 아니다.
923 번째 평점리뷰: 예고편이 거의 영화내용이긴한데 bgm이나 영상기법이 되게 몰입되게 만들어줘요  
분류결과: 스포 X
다음 review는 spo:6.74494292324949e-32 <nonspo:9.796656524925279e-29이므로 스포가 아니다.
924 번째 평점리뷰: 생각보다 재밌었지만 징그러웠다 따흑  
분류결과: 스포 X
다음 review는 spo:2.4199315653905327e-15 <nonspo:1.878454835465414e-14이므로 스포가 아니다.
925 번째 평점리뷰: 보다보면 혹시 이러지 않을까? 했던게 실제로 일어남. 끝나고 주변 사람을 다시 한번 훑어보게 됨 ㅋㅋ  
분류결과: 스포 X
다음 review는 spo:3.655795622357447e-34 <nonspo:3.998635316296033e-31이므로 스포가 아니다.
926 번째 평점리뷰: 귀신공포보다 무섭게 심리를 건드리는 스릴러가 여기있네  
분류결과: 스포 X
다음 review는 spo:1.3358686280769632e-25 <nonspo:6.731340398923103e-22이므로 스포가 아니다.
927 번째 평점리뷰: 대다나다.. 모든 복선들이 다 연결이 되다니요~b  
분류결과: 스포 X
다음 review는 spo:3.423301125181119e-18 <nonspo:2.7802922608287587e-16이므로 스포가 아니다.
928 번째 평점리뷰: 침전의 방 속에서 고통 받고 있었을 영혼들을 생각하면 더 소름 돋는 영화  
분류결과: 스포 X
다음 review는 spo:4.587624057789094e-24 <nonsp

956 번째 평점리뷰: 솔직히 말해서 한국 영화 보단 재미있지ㅋㅋ  
분류결과: 스포 X
다음 review는 spo:8.268099515084321e-15 <nonspo:7.266654231931994e-13이므로 스포가 아니다.
957 번째 평점리뷰: 신선한 소재... 정말 그거 하나뿐이다. 친구의 하드캐리 제외.  
분류결과: 스포 X
다음 review는 spo:1.2955225736729201e-31 <nonspo:1.4688320395194104e-30이므로 스포가 아니다.
958 번째 평점리뷰: 뭐..다들 아는 그것에서  멈춤니다ㅋ뭐 엄청난 반전은 아예없음  쓸더없이 평이높아요  조조로 보긴딱좋음ㅎ돈자주고 보믄 바보짓  
분류결과: 스포 X
다음 review는 spo:1.3436157562149319e-64 <nonspo:7.465487739328949e-62이므로 스포가 아니다.
959 번째 평점리뷰: 예고편이 끝인 영화..신선하다며 띄워줬는데 B급 그 이상 그 이하도 아닌 영화  
분류결과: 스포 X
다음 review는 spo:7.263492366213178e-37 <nonspo:3.6892620258786464e-35이므로 스포가 아니다.
960 번째 평점리뷰: 평소 스릴러를 즐겨보지 않는 분들에게는 평점이 높을 것같아요 하지만 저처럼 스릴러 매니아에게는 그저 평범한 영화 중 1편 일뿐 뻔한 스토리에 크게 감흥은 없네요  
분류결과: 스포 X
다음 review는 spo:4.3268756778668616e-61 <nonspo:9.853557407206344e-57이므로 스포가 아니다.
961 번째 평점리뷰: 이 영하 터질만 하다. 왜 진작 개봉 안했나  
분류결과: 스포 X
다음 review는 spo:1.1306233423819596e-29 <nonspo:7.00132118570094e-28이므로 스포가 아니다.
962 번째 평점리뷰: 예상가능하면서도 이렇게 재밌는 영화는 처음이다 ㅎㅎ  
분류결과: 스포 X
다음 review는 spo:4.3019

990 번째 평점리뷰: 관람객영화 재밌게 봤습니다 너무 과장되지도 억지스럽지도 않게 스토리가 이어져서 시간 가는줄 모르고 봤네요 ~  
분류결과: 스포 X
다음 review는 spo:3.180526676686703e-35 <nonspo:6.37734710360552e-33이므로 스포가 아니다.
991 번째 평점리뷰: 존 말코비치 되기 다운그레이드 버전  
분류결과: 스포 O
다음 review는 spo:4.035579201656434e-24 >nonspo:3.1765194665513645e-24이므로 스포이다.
992 번째 평점리뷰: 관람객너무 재밌게 봤습니다 ㅎㅎ  
분류결과: 스포 X
다음 review는 spo:8.553248117872838e-12 <nonspo:3.5210931470235884e-10이므로 스포가 아니다.
993 번째 평점리뷰: 신선하긴 한데 전개가 너무 뻔함  고퀄킬링타임용  
분류결과: 스포 X
다음 review는 spo:2.5599361056258823e-36 <nonspo:1.8494991600144903e-34이므로 스포가 아니다.
994 번째 평점리뷰: 다들 경찰차엔 백인이 타고 있었을 거라고 생각했을걸.  
분류결과: 스포 X
다음 review는 spo:1.5464192282818175e-15 <nonspo:8.822794107696593e-15이므로 스포가 아니다.
995 번째 평점리뷰: 관람객영화 끝난 후 장면 하나하나 곱씹을때마다 소름돋았어여..쫄보에게는 보기 힘든 영화였지만 후회하지는 않습니당 다들 보세영  
분류결과: 스포 X
다음 review는 spo:5.263946961473019e-46 <nonspo:5.315410336418379e-44이므로 스포가 아니다.
996 번째 평점리뷰: 솔직히 점수엔 후한 편인데.. 보시면서 반전 예상하시면 아마 예상하시는대로 전부 맞을겁니다 나름 신선한걸 들고 나와서 기대하고 봤는데 오~ 할만한건 배우들 연기력뿐.. 생소한 얼굴들이 많은데 연기는 정말 소름 끼치더군요 반전은 소재에

# 영화 부산행

In [6]:
file_name = "부산행_최종본.csv"
uptitle= "부산행_상위tf.csv"
ctitle = "부산행_네이버평점.csv" 

csv_file = open(file_name, 'r')
doc = csv.reader(csv_file)
next(doc) # header 넘어가기
    
# 문장별
sentences = []
for line in doc:
    sentences.extend(split_sentence(line))
# 형태소별    
sentence_morphs = []
for s in sentences:
        sentence_morphs.append(preprocessing(s))    

for a in sentence_morphs:
    if a == []:
        sentence_morphs.remove(a)    

# 상위tf 2차 전처리   
f=open(uptitle,'r')
rf=csv.reader(f)
next(rf)
TF=[]
for row in rf:
    TF.append(row[0])
stop=['이', '을', '스', '여','노', '은', '아', '터', '드', '마', '자', '고', '레','거','개방','관람','씬','내용','결국','그런데',
      '거','늘', '의', '라', '누', '어', '건','진','자', '마','로','스','까','은','시','드','곽','왜','그리하','약간','갑자기',
      '마','저','거','라','까','분','늘','자','의','어','스','여','을','몇','몇몇','걸','시','드','단','서','인','블록버스터',
      '늘','메','파','자','김','티','리','바','스','건','대','새','어','보','시','디','르','을','고','예고편','스포일러','장르',
       '어쩌','듯하','경','어느','결말','이때','재미있','그리하','이번','조금','스','자','누','터','라','뜨','다','막','분','임',
      '위','하다','리','사','유','면','감독','스포','줄거리','영화']
newTF=[]
for word in TF:
    if word not in stop:
        newTF.append(word)
    newTF=[]
    for word in TF:
        if word not in stop:
            newTF.append(word)   

dataset2=[]
for i in range(len(sentences)):
    a=split_morphs(sentences[i])
    dataset= []
    for w in a:
        if w in newTF: 
            dataset.append(w)
    dataset2.append(dataset)
    
dataset3=[]
for k in range(len(dataset2)):
    if dataset2[k] != []:
        dataset3.append(dataset2[k])
 ############################ Apriori 알고리즘 적용 ################################
te = TransactionEncoder()
te_ary = te.fit(dataset3).transform(dataset3)
df = pd.DataFrame(te_ary, columns=te.columns_) #위에서 나온걸 보기 좋게 데이터프레임으로 변경
    
frequent_itemsets3 = apriori(df, min_support=0.003, max_len=3, use_colnames=True)
frequent_itemsets2 = apriori(df, min_support=0.003, max_len=2, use_colnames=True)

frequentThree=frequent_itemsets3[len(frequent_itemsets2):len(frequent_itemsets3)]
frequentThree=frequentThree.sort_values("support",ascending=False)

# 일정 확률 이상의 데이터 선택
frequentThree=frequentThree[frequentThree["support"]>=0.0049]
# frozenset를 list로 변화
sets=[]
for line in frequentThree['itemsets']:
    sets.append(line)
association=[list(x) for x in sets]

In [7]:
############################### 나이브 베이지안 알고리즘 적용 ################################
spo = []
nonspo = []

for row in sentence_morphs:
    s1 = set(row)
    for dataset in association:
        s2 = set(dataset)
        isSpo = False
        if s2.issubset(s1) == True:
            spo.append(row)
            isSpo = True
            break
    if not isSpo:
        nonspo.append(row)
        
csv_file = open(ctitle, 'r')
doc = csv.reader(csv_file)
next(doc) # header 넘어가기

review_token = []
totalreview = []
for line in doc:
    text = line[2]
    totalreview.append(text) # totalreview 리스트에 결과 저장
    
index = 0
for comment in totalreview[0:1000]:
    review=preprocessing(comment) # 형태소 분석
    index+=1
    
    
    #__________prior probabilities
    prior_spo=len(spo)/(len(spo)+len(nonspo))
    prior_nonspo=len(nonspo)/(len(spo)+len(nonspo))
    
    #_________training data
    
    #spo의 중복단어 리스트 (the sum of term frequency for class SPO)
    spo_overlap=[]
    for i in range(len(spo)):
        spo_overlap.extend(spo[i])
        
    #nonspo의 중복단어 리스트 (the sum of term frequency for class NONSPO)
    nonspo_overlap=[]   
    for i in range(len(nonspo)):
        nonspo_overlap.extend(nonspo[i])
    
    #중복없는 전체단어 갯수: Vocabulary
    all_spo=[]
    for i in range(len(spo)):
        for j in range(len(spo[i])):
            if(spo[i][j] not in all_spo):
                all_spo.append(spo[i][j])
                
    all_nonspo=[]
    for i in range(len(nonspo)):
        for j in range(len(nonspo[i])):
            if(nonspo[i][j] not in all_nonspo):
                all_nonspo.append(nonspo[i][j])
    
    overall=[]
    overall.extend(all_spo)
    overall.extend(all_nonspo)
    
    nonover=[]
    for i in range(len(overall)):
        if(overall[i] not in nonover):
            nonover.append(overall[i])
    all_count=len(nonover)
    
    
    #___________likelihood
    #spo의  likelihood
    spo_like_result=1
    for i in range(len(review)):
        count=1# Laplace smoothing
        for j in range(len(spo_overlap)):
            if (review[i]==spo_overlap[j]):
                count=count+1
        spo_like_result=spo_like_result*(count/(len(spo_overlap)+all_count))#review 단어별 like
        
    
    #nonspo의  likelihood
    nonspo_like_result=1
    for i in range(len(review)):
        ncount=1# Laplace smoothing
        for j in range(len(nonspo_overlap)):
            if (review[i]==nonspo_overlap[j]):
                ncount=ncount+1
        nonspo_like_result=nonspo_like_result*(ncount/(len(nonspo_overlap)+all_count))#review 단어별 like 
        
    #_________posterior probabilities
    
    #P(SPO | REVIEW)=p(REVIEW|SPO)P(SPO)
    post_spo=spo_like_result*prior_spo
    
    #P(NONSPO | REVIEW)=P(REVIEW|NONSPO)P(NONSPO)
    post_nonspo=nonspo_like_result*prior_nonspo
    if(post_spo>post_nonspo):
        print(index,"번째 평점리뷰:",comment)
        print("분류결과: 스포 O")
        print("다음 review는 spo:{} >nonspo:{}이므로 스포이다.".format(post_spo,post_nonspo))
        print("=============================================================================================")
    
    elif(post_spo<post_nonspo):
        print(index,"번째 평점리뷰:",comment)
        print("분류결과: 스포 X")
        print("다음 review는 spo:{} <nonspo:{}이므로 스포가 아니다.".format(post_spo,post_nonspo))
        print("=============================================================================================")

        

1 번째 평점리뷰: 좀비가 무서운게아니라 사람이무섭다는걸 알려주는영화  
분류결과: 스포 X
다음 review는 spo:7.606615683475457e-15 <nonspo:1.1912692576763776e-13이므로 스포가 아니다.
2 번째 평점리뷰: 관람객그냥 한국형 좀비물이 어색하지 않았다는 거에 큰 점수를 주며 앞으로의 한국영화를 기대해봅니다.  
분류결과: 스포 X
다음 review는 spo:1.867178218422833e-34 <nonspo:3.2568567209419635e-32이므로 스포가 아니다.
3 번째 평점리뷰: 버스회사 상무라는 사람이 진심 역대급 민폐캐릭터인듯개짜증나고 생존력도 좋음  
분류결과: 스포 X
다음 review는 spo:5.765292602890253e-42 <nonspo:3.72927299118499e-39이므로 스포가 아니다.
4 번째 평점리뷰: 관람객생각보다 너무 재밌었다  
분류결과: 스포 X
다음 review는 spo:2.3653709813760574e-12 <nonspo:2.4530217981656015e-10이므로 스포가 아니다.
5 번째 평점리뷰: 영화가 너무 비현실적이네요. 예를들어 공유 얼굴이라던가 공유 얼굴 같은 부분이...  
분류결과: 스포 X
다음 review는 spo:2.923639734201557e-38 <nonspo:5.829928754140664e-37이므로 스포가 아니다.
6 번째 평점리뷰: 관람객마동석 진짜 웃겨요 사랑해요  
분류결과: 스포 X
다음 review는 spo:5.473443424071444e-20 <nonspo:2.9168629131785185e-19이므로 스포가 아니다.
7 번째 평점리뷰: 배급사는 스포일러 법적대응 해야할듯  
분류결과: 스포 X
다음 review는 spo:3.7118116837336915e-20 <nonspo:6.770152465496949e-19이므로 스포가 아니다.
8 번째 평점리뷰: 관람객한국에서 볼 수 없었던 영화  
분류결과: 스포 X
다음 

35 번째 평점리뷰: 관람객기대없이 봐서 재밌게 봤어요 영화내용이 이런 사건이 생긴 이유나 해결방법 등 전혀 없이 그냥 계속 쫓기고 죽다가 끝나서 조금 허무하긴 하지만 스릴넘치네요  
분류결과: 스포 X
다음 review는 spo:3.940627513998256e-76 <nonspo:1.0315524560035465e-71이므로 스포가 아니다.
36 번째 평점리뷰: 아 존나 신선했는데 짜증나고 답답한 느낌이 가실질 않는다 이걸노렸나?  
분류결과: 스포 X
다음 review는 spo:1.6795255774461618e-34 <nonspo:1.1762786164084038e-33이므로 스포가 아니다.
37 번째 평점리뷰: Kt 여승무원을  소희역 시키지  훨 이쁘고 연기도 잘하든데 난친구가 못가서 대신보고온건데..이영화 배우들 한테 연기상 줄려거든좀비들 한테 주길  
분류결과: 스포 X
다음 review는 spo:3.100473922176254e-59 <nonspo:1.8233722113539143e-55이므로 스포가 아니다.
38 번째 평점리뷰: 관람객마지막 아이의 노래를 듣는순간 짜증나서 내가 좀비로 변할것 같았다  
분류결과: 스포 X
다음 review는 spo:3.0942390345733165e-32 <nonspo:6.067184129840974e-32이므로 스포가 아니다.
39 번째 평점리뷰: 관람객원래 sf나 좀비 같은 실제하지않는 허구의 영화를 좋아해서 별점은 높다. 실제로 영화도 재밌다. 그러나 아쉬운건 영화 초반 감염경로나 마지막 결말부분이 너무 부족한게아닌가...  
분류결과: 스포 X
다음 review는 spo:1.499842038920629e-75 <nonspo:9.7493504844203e-70이므로 스포가 아니다.
40 번째 평점리뷰: 이성적인 판단이 가능한 사람은 내 별점 공감할것이다  
분류결과: 스포 X
다음 review는 spo:3.7118116837336915e-20 <nonspo:2.4823892373488807e-18이므로 

67 번째 평점리뷰: 관람객진짜 마동석말고는 하....스토리라는게 없고 허구한날 눈물찔찔하려고 만든영화  
분류결과: 스포 X
다음 review는 spo:2.0343755407088782e-33 <nonspo:2.2617060562096964e-32이므로 스포가 아니다.
68 번째 평점리뷰: 관람객입니다. 태어나서 이런 후기 처음써보는데 정말 애국심,팬심 다버리고 적날하게씀. 아역,아역이부르는노래,스토리,전개,안소희 <- 진짜 다음날 눈떴는데도 미치게만듬...좀비왜생긴지,해결은어떻게된건지,아역이부르는노래는트로트임그거? 안소희연기...  
분류결과: 스포 X
다음 review는 spo:5.108357019478277e-128 <nonspo:1.7662715079559445e-125이므로 스포가 아니다.
69 번째 평점리뷰: 관람객왜 칸느에서 기립박수를쳤는지 전혀이해가 가지않는다  
분류결과: 스포 X
다음 review는 spo:2.9860565162344874e-31 <nonspo:5.255612858112747e-29이므로 스포가 아니다.
70 번째 평점리뷰: 마동석 없으면 망할 영화... 영화을 제대로 만들고 스포 신고하던가해라 배우들 연기에 할밀이 없다  
분류결과: 스포 X
다음 review는 spo:2.9725028116960936e-47 <nonspo:4.412526257184587e-46이므로 스포가 아니다.
71 번째 평점리뷰: 다들 알바인가요. 세상에 아주 재미없습니다. 진짜 나가려다 참았다. 마동석하드캐리영화.  
분류결과: 스포 X
다음 review는 spo:5.551749009817692e-45 <nonspo:6.145729427912463e-42이므로 스포가 아니다.
72 번째 평점리뷰: 후반부부터 우리나라 영화 전통의 질질짜는 감성팔이 + 뻔한 사회풍자 + 생각보다 작은 스케일 +외국영화 표절 + 김치드라마에다가 좀비물 어색하게 합쳐논 느낌 + 노잼 =평점 1점  
분류결과: 스포 X
다음 review는 spo:1.406392006401

101 번째 평점리뷰: 딱 5분 고라니가 치여서 비보잉하며 일어날때까진 좋았다 그 후 110분간 주구장창 좀비의 크어어어엉을 듣다가 축농증가능성이 농후한 아이의 노래를 들으며 끝나버렸다. 일단 좀비 발생과 확산 과정등 필요한 장면이 모두 짤려버렸기에 이해는 포기하는게 좋다  
분류결과: 스포 X
다음 review는 spo:4.955464230961097e-114 <nonspo:2.3926045236183594e-112이므로 스포가 아니다.
102 번째 평점리뷰: 하....새로운시도는좋았다만 이건 뭐.. 10 no 잼  
분류결과: 스포 X
다음 review는 spo:2.8508940818084996e-15 <nonspo:5.095072879793111e-14이므로 스포가 아니다.
103 번째 평점리뷰: 소희 연기 형편 없었고 제발 소희가 빨리 감염돼서 프레임 아웃을 바랐다. 그 아역 왜 썼는지 모르겠다. 노래가 처량하게 들리지도 않았고 초반부터 거슬리던 말투 표정 발음.. 연출도 형편없었고 초반 마동석이 하드캐리. 이후 폭망.  
분류결과: 스포 X
다음 review는 spo:3.280657332301846e-113 <nonspo:1.5172745165033956e-110이므로 스포가 아니다.
104 번째 평점리뷰: 조기종영행 다시는 평식이 형 평점에 속지 않으리  
분류결과: 스포 X
다음 review는 spo:3.822450946431768e-28 <nonspo:4.484424665807731e-28이므로 스포가 아니다.
105 번째 평점리뷰: 관람객일단 재미가없다. 개연성도없다. 연기를못하는 배우떄문에 몰입도도 떨어진다.  
분류결과: 스포 X
다음 review는 spo:1.5740386182815094e-34 <nonspo:1.7027814484564287e-33이므로 스포가 아니다.
106 번째 평점리뷰: 평점보니 딱 알겠네... 언플만 열라 열심히하고 까보니 개발암 덩어리? 발연기와 발암 꼬맹이는 극의 집중을 방해한다는거 모르나?? 결국엔 이 영화도

133 번째 평점리뷰: 관람객억지신파 전형적 옛날한국영화  
분류결과: 스포 X
다음 review는 spo:2.813323896573781e-25 <nonspo:3.632383979304262e-24이므로 스포가 아니다.
134 번째 평점리뷰: 살다살다 이딴 20년전 좀비영화 보다 못한 망작을 재난 블록버스터로 포장하고 ㅉㅉㅉ  영화 개떡같이 만들고 평점알바 언플 적당히 하면 떼돈번다는  국민을 개돼지로 보는 이런 잡영화는 망해야 한다 ㅉㅉㅉ 가족끼리 가서 돈 시간 기분까지 잡치고ㅉㅉㅉ  
분류결과: 스포 X
다음 review는 spo:5.955109981986763e-112 <nonspo:3.2745373083878857e-110이므로 스포가 아니다.
135 번째 평점리뷰: 이렇게 한바탕 난리굿 떠는 소동영화들 다신 보지 말아야지~  
분류결과: 스포 X
다음 review는 spo:3.8683203577889487e-25 <nonspo:6.65937062872448e-25이므로 스포가 아니다.
136 번째 평점리뷰: 관람객어마어마한 쓰레기 영화를 만들어놓음, 왜등장하는지 모르는 케릭터들 하며,, 개연성없는스토리하며... 결말도 ...  
분류결과: 스포 X
다음 review는 spo:2.1763331327225855e-45 <nonspo:3.5765146107484094e-43이므로 스포가 아니다.
137 번째 평점리뷰: 괜히 봤다..  시작과 끝이 없는 영화, 즉 내용없는 영화다.. 마동석 제외하고는 연기들도 어색하고, 답없는 상업성만 강조한 삼류영화. 진짜 보지마세요 ㅋㅋ 인천상륙작전이 걱정되네요..  
분류결과: 스포 X
다음 review는 spo:3.4392257841868296e-75 <nonspo:8.458652381563665e-71이므로 스포가 아니다.
138 번째 평점리뷰: 월드워z 많이 보고 배낀듯 마동석팔에 테이프감고 좀비 격투씬좀비들이 탑쌓고 쫒아오는씬초반에 볼거리는 많았으나 전반적으로 스토리가 끈어지는 느낌에 중반쯤 지루함을 떨칠수가 

...

843 번째 평점리뷰: 영화에 등장하는 모든 인물의 의식이 지나치게 1차원적이다. 중간에 지하철 문열어서 최고의플레이 기록한 할머니의 죽음이 감동적인 브금과 함께 연출되는 장면에서는 진짜로 쌍욕을 내뱉었다. 극단적인 신파성으로 점철된 한국 좀비 영화  
분류결과: 스포 X
다음 review는 spo:1.0944856642282352e-108 <nonspo:5.729507524920973e-105이므로 스포가 아니다.
844 번째 평점리뷰: 억지영화...그게 다다  
분류결과: 스포 X
다음 review는 spo:4.663159934712799e-11 <nonspo:4.599415871560503e-10이므로 스포가 아니다.
845 번째 평점리뷰: 진심으로 재미 없었음. 개연성 없고 막흘러가는 무대뽀식 영화 '2012' 다음으로 재미없었음. 최악수준  
분류결과: 스포 X
다음 review는 spo:9.90550770914909e-44 <nonspo:1.881612078088354e-42이므로 스포가 아니다.
846 번째 평점리뷰: 마지막에 정유미 총 맞고 죽었으면 그자리에서 일어나 개욕했을뻔.마동석빼고 볼게 없음. 보는내내 고구마 10개 한꺼번에 먹은듯. 초반부터 이상해보였던 노숙자도 실마리를 풀어줄지 알았지만 그냥 희생양에 불과하고 전반적으로 생존을 위해 희생만 강조함.  
분류결과: 스포 X
다음 review는 spo:1.6826930503258257e-101 <nonspo:1.220571074179668e-100이므로 스포가 아니다.
847 번째 평점리뷰: 헬좆센 좀비물이라는 점에서는 박수를 쳐주지만 애새끼 때문에 암 생길뻔 함. 들창코에 남자인줄알았다  
분류결과: 스포 O
다음 review는 spo:1.4693275285790619e-49 >nonspo:1.0664958288229524e-50이므로 스포이다.
848 번째 평점리뷰: 재미없고 연기도 별로임 에구 내 시간 ㅜ  
분류결과: 스포 X
다음 review는 spo:5.56771752560053

875 번째 평점리뷰: 재미도 없고 감동도 없는 영화  
분류결과: 스포 X
다음 review는 spo:4.663159934712798e-11 <nonspo:4.599415871560503e-10이므로 스포가 아니다.
876 번째 평점리뷰: 난리나게놀라운영화입니다  
분류결과: 스포 X
다음 review는 spo:8.196320485199436e-15 <nonspo:4.6791485630753067e-14이므로 스포가 아니다.
877 번째 평점리뷰: 정말 뻔한 내용에 재미라고는 하나도 없고 결말도 없고소희 그년은 왜 나와서 연기하는거야?? 진짜 검증안된 아이돌들 출연좀 시키지마라  
분류결과: 스포 X
다음 review는 spo:1.6515998031842125e-58 <nonspo:2.390835014066377e-55이므로 스포가 아니다.
878 번째 평점리뷰:  외국영화따라한 할말이 없는 영화  
분류결과: 스포 X
다음 review는 spo:1.8851537115958704e-13 <nonspo:1.2633701120303329e-11이므로 스포가 아니다.
879 번째 평점리뷰: 엉청난 기대를 하고 본영화지만상당히내용불친절한 B급영화짜증유발 시키는 아역배우 몰입도 떨어지는 배우억지로 스토리를 끌고나가는 듯한 후반부까지 안봤으면 좋았을 기대만 하고 있었음 좋았을 영화  
분류결과: 스포 X
다음 review는 spo:5.298661792752714e-87 <nonspo:5.931546022181319e-84이므로 스포가 아니다.
880 번째 평점리뷰: 좀비분장은 부산행... 스토리는 안드로메다행...  
분류결과: 스포 O
다음 review는 spo:4.8441882619620335e-20 >nonspo:2.3869374526355317e-20이므로 스포이다.
881 번째 평점리뷰: 억지감동발연기진부한스토리뻔한전개어쩜 하나도 안비껴가는지이게 재미있다니 헐좀비연기까지 구린건 요근래 첨봤음마요미,공유,악역하신분 세분빼곤 하... 진짜 캐스팅 미스  
분류결과

907 번째 평점리뷰: ....;;; 월드워Z를 중국짝퉁시장에서... 그것도 만들다가 중간에 포기해버린... 짝퉁영화같음...  
분류결과: 스포 X
다음 review는 spo:5.53543077864214e-45 <nonspo:1.0117123765456684e-42이므로 스포가 아니다.
908 번째 평점리뷰: 남자앤지 여자앤지도 모르겠고 왠지 전쟁터에 나올것같은 애를 집어넣었네... 감독친척인가...다른 배우들은 괜찮은것 같은데...어쨋든 감정이입이안됨..이름도수완인가...  치마입은거보고여자앤줄암.예매율1위라길래봣더니 괜히봣네...  
분류결과: 스포 O
다음 review는 spo:4.597482709969946e-117 >nonspo:1.2402195891701947e-117이므로 스포이다.
909 번째 평점리뷰: 한국 사회의 이기적인 면과 뉴스는 믿을 수 없다 라는 것만 보여준 영화, 왜 바이러스 감염 원인과 핵신은 어디감? 핵신이 없더라도 결말이 왜 이럼..  
분류결과: 스포 X
다음 review는 spo:7.380502154220406e-70 <nonspo:3.420490767054506e-65이므로 스포가 아니다.
910 번째 평점리뷰: 저에겐 최악중최악. 암 유발 쉬운 분들은 이 영화 보지 마세요^^  
분류결과: 스포 X
다음 review는 spo:6.362834137961811e-34 <nonspo:3.5821768647846505e-32이므로 스포가 아니다.
911 번째 평점리뷰: 하....이십.새끼 폭스사같은 영화  
분류결과: 스포 X
다음 review는 spo:2.1856854627198494e-14 <nonspo:8.422467413535553e-14이므로 스포가 아니다.
912 번째 평점리뷰:  여기 출연한 사람들이 전문배우인지조차 의심스럽다. 하나부터 열까지 모두 발연기에 상황연출도 어처니구없이 유치하다. 3~5점 정도가 적당하지만 평점조절을 위해 1점만준다  
분류결과: 스포 X
다음 review는 spo:4.55699730

938 번째 평점리뷰: 관람객노잼입니다 별로 비추천  한국영화 클라스  
분류결과: 스포 X
다음 review는 spo:5.658115813402525e-35 <nonspo:2.863848577668122e-34이므로 스포가 아니다.
939 번째 평점리뷰: 공유회상씬과 할머니문열기시전씬은 어이가 없어서 현웃이ㅋㅋㅋㅋ그리고마지막 꼬마의 노래씬은 정말...ㅎㅎ...인간인걸 알리려는 의도였겠지만 내눈엔 좀비모여라~~~죽을려고 환장한걸로 밖에 안보였음.  진짜 개연성과 기승전결따위 밥말아 먹은 영화 돈이 아깝다  
분류결과: 스포 X
다음 review는 spo:1.3245638818125294e-132 <nonspo:1.0958885264827602e-130이므로 스포가 아니다.
940 번째 평점리뷰: 진짜 기대안하고 봐도 재미없다 재미가 없으면 내용이 시원하게 이어가든가 답답하고 암걸리네요  그리고 연기는 왜이리 어색해 좀비영화가 아니라 감성자극 영화인지 쓸데없는 감성팔이 ,  
분류결과: 스포 X
다음 review는 spo:3.4489678413929094e-81 <nonspo:1.1932663895209318e-77이므로 스포가 아니다.
941 번째 평점리뷰: 그냥 말이필요없다 재미없다  
분류결과: 스포 O
다음 review는 spo:4.054921682358956e-12 >nonspo:2.981102879715141e-12이므로 스포이다.
942 번째 평점리뷰: 이게 왜 평이 높은지 이해가 안간다 알바아님? 시간아깝게 이거 보지말고 차라리 월드워z 한번 더 볼껄 봐도봐도 재밌는데  
분류결과: 스포 X
다음 review는 spo:5.1945922727542585e-52 <nonspo:1.3163605658614726e-47이므로 스포가 아니다.
943 번째 평점리뷰: 140자만 되서 짧게 씁니다.보기전 미드 워킹데드 보세요,보고나면 이영화는 쓰레기 입니다.서론 본론 결말 아무것도 없습니다.영화 진행.. 답답합니다.좀비도 말도 안되게 바로 변신. 근데 물린 공

970 번째 평점리뷰: 와 진심 재미없다....천만이라길래 기대하고 구입해서 봤는데.이건뭐 미국 삼류 좀비영화보다 재미 없다; 해도해도 너무한다;;  
분류결과: 스포 X
다음 review는 spo:4.2058704619830996e-45 <nonspo:9.097943778416201e-45이므로 스포가 아니다.
971 번째 평점리뷰: 좀비나올때는 볼만한데 사람들하는게 현실성 더럽게 없네  
분류결과: 스포 X
다음 review는 spo:4.355532624985207e-15 <nonspo:3.161024807055318e-14이므로 스포가 아니다.
972 번째 평점리뷰: 부끄럽다. 한국 관객의 수준이...천만 영화ㅋㅋ  
분류결과: 스포 X
다음 review는 spo:1.0244600247104989e-17 <nonspo:1.5537499908315498e-15이므로 스포가 아니다.
973 번째 평점리뷰: 너무 비현실적이어서, 오히려 버스회사 상무가 더 현실적인 영화. 사람들 구하지 않는다고 아빠 타박하는 딸, 위급 상황에서 남자친구 왔다고 문열라는 안소희, 사람 다죽어가는데  어조 고저 차이 없는 기차장, 최악의 빌런 할머니, 등등등 발암*1000  
분류결과: 스포 O
다음 review는 spo:2.055409799345279e-115 >nonspo:9.517584160383396e-117이므로 스포이다.
974 번째 평점리뷰: 관람객별로....쫌 아닌듯  
분류결과: 스포 X
다음 review는 spo:7.91915022724583e-16 <nonspo:2.079621583589025e-15이므로 스포가 아니다.
975 번째 평점리뷰: 몰입도 제로. 스토리 엉망  
분류결과: 스포 X
다음 review는 spo:7.91915022724583e-16 <nonspo:8.318486334356101e-16이므로 스포가 아니다.
976 번째 평점리뷰: 네이버 알바의 힘인가...별다섯개가 천지내..소희야  노래도 못하지만  그냥 노래해라 연기또하면암걸리겠다., 

# 영화 서치

In [5]:
file_name = "서치_최종본.csv"
uptitle= "서치_상위tf.csv"
ctitle = "서치_네이버평점.csv" 

csv_file = open(file_name, 'r')
doc = csv.reader(csv_file)
next(doc) # header 넘어가기
    
# 문장별
sentences = []
for line in doc:
    sentences.extend(split_sentence(line))
# 형태소별    
sentence_morphs = []
for s in sentences:
        sentence_morphs.append(preprocessing(s))    

for a in sentence_morphs:
    if a == []:
        sentence_morphs.remove(a)    

# 상위tf 2차 전처리   
f=open(uptitle,'r')
rf=csv.reader(f)
next(rf)
TF=[]
for row in rf:
    TF.append(row[0])
stop=['이', '을', '스', '여','노', '은', '아', '터', '드', '마', '자', '고', '레','거','개방','관람','씬','내용','결국','그런데',
      '거','늘', '의', '라', '누', '어', '건','진','자', '마','로','스','까','은','시','드','곽','왜','그리하','약간','갑자기',
      '마','저','거','라','까','분','늘','자','의','어','스','여','을','몇','몇몇','걸','시','드','단','서','인','블록버스터',
      '늘','메','파','자','김','티','리','바','스','건','대','새','어','보','시','디','르','을','고','예고편','스포일러','장르',
       '어쩌','듯하','경','어느','결말','이때','재미있','그리하','이번','조금','스','자','누','터','라','뜨','다','막','분','임',
      '위','하다','리','사','유','면','감독','스포','줄거리','영화']
newTF=[]
for word in TF:
    if word not in stop:
        newTF.append(word)
    newTF=[]
    for word in TF:
        if word not in stop:
            newTF.append(word)   

dataset2=[]
for i in range(len(sentences)):
    a=split_morphs(sentences[i])
    dataset= []
    for w in a:
        if w in newTF: 
            dataset.append(w)
    dataset2.append(dataset)
    
dataset3=[]
for k in range(len(dataset2)):
    if dataset2[k] != []:
        dataset3.append(dataset2[k])
 ############################ Apriori 알고리즘 적용 ################################
te = TransactionEncoder()
te_ary = te.fit(dataset3).transform(dataset3)
df = pd.DataFrame(te_ary, columns=te.columns_) #위에서 나온걸 보기 좋게 데이터프레임으로 변경
    
frequent_itemsets3 = apriori(df, min_support=0.003, max_len=3, use_colnames=True)
frequent_itemsets2 = apriori(df, min_support=0.003, max_len=2, use_colnames=True)

frequentThree=frequent_itemsets3[len(frequent_itemsets2):len(frequent_itemsets3)]
frequentThree=frequentThree.sort_values("support",ascending=False)

# 일정 확률 이상의 데이터 선택
frequentThree=frequentThree[frequentThree["support"]>=0.007]

# frozenset를 list로 변화
sets=[]
for line in frequentThree['itemsets']:
    sets.append(line)
association=[list(x) for x in sets]

In [6]:
############################### 나이브 베이지안 알고리즘 적용 ################################
spo = []
nonspo = []

for row in sentence_morphs:
    s1 = set(row)
    for dataset in association:
        s2 = set(dataset)
        isSpo = False
        if s2.issubset(s1) == True:
            spo.append(row)
            isSpo = True
            break
    if not isSpo:
        nonspo.append(row)
        
csv_file = open(ctitle, 'r')
doc = csv.reader(csv_file)
next(doc) # header 넘어가기

review_token = []
totalreview = []
for line in doc:
    text = line[2]
    totalreview.append(text) # totalreview 리스트에 결과 저장
    
index = 0
for comment in totalreview[0:1000]:
    review=preprocessing(comment) # 형태소 분석
    index+=1
    
    
    #__________prior probabilities
    prior_spo=len(spo)/(len(spo)+len(nonspo))
    prior_nonspo=len(nonspo)/(len(spo)+len(nonspo))
    
    #_________training data
    
    #spo의 중복단어 리스트 (the sum of term frequency for class SPO)
    spo_overlap=[]
    for i in range(len(spo)):
        spo_overlap.extend(spo[i])
        
    #nonspo의 중복단어 리스트(the sum of term frequency for class NONSPO)
    nonspo_overlap=[]   
    for i in range(len(nonspo)):
        nonspo_overlap.extend(nonspo[i])
    
    #중복없는 전체단어 갯수: Vocabulary
    all_spo=[]
    for i in range(len(spo)):
        for j in range(len(spo[i])):
            if(spo[i][j] not in all_spo):
                all_spo.append(spo[i][j])
                
    all_nonspo=[]
    for i in range(len(nonspo)):
        for j in range(len(nonspo[i])):
            if(nonspo[i][j] not in all_nonspo):
                all_nonspo.append(nonspo[i][j])
    
    overall=[]
    overall.extend(all_spo)
    overall.extend(all_nonspo)
    
    nonover=[]
    for i in range(len(overall)):
        if(overall[i] not in nonover):
            nonover.append(overall[i])
    all_count=len(nonover)
    
    #_________likelihood
    #spo의  likelihood
    spo_like_result=1
    for i in range(len(review)):
        count=1# Laplace smoothing
        for j in range(len(spo_overlap)):
            if (review[i]==spo_overlap[j]):
                count=count+1
        spo_like_result=spo_like_result*(count/(len(spo_overlap)+all_count))#review 단어별 like
        
    
    #nonspo의  likelihood
    nonspo_like_result=1
    for i in range(len(review)):
        ncount=1# Laplace smoothing
        for j in range(len(nonspo_overlap)):
            if (review[i]==nonspo_overlap[j]):
                ncount=ncount+1
        nonspo_like_result=nonspo_like_result*(ncount/(len(nonspo_overlap)+all_count))#review 단어별 like 
        
    #_________posterior probabilities
    
    #P(SPO | REVIEW)=p(REVIEW|SPO)P(SPO)
    post_spo=spo_like_result*prior_spo
    
    #P(NONSPO | REVIEW)=P(REVIEW|NONSPO)P(NONSPO)
    post_nonspo=nonspo_like_result*prior_nonspo
    if(post_spo>post_nonspo):
        print(index,"번째 평점리뷰:",comment)
        print("분류결과: 스포 O")
        print("다음 review는 spo:{} >nonspo:{}이므로 스포이다.".format(post_spo,post_nonspo))
        print("=============================================================================================")
    
    elif(post_spo<post_nonspo):
        print(index,"번째 평점리뷰:",comment)
        print("분류결과: 스포 X")
        print("다음 review는 spo:{} <nonspo:{}이므로 스포가 아니다.".format(post_spo,post_nonspo))
        print("=============================================================================================")

        

1 번째 평점리뷰: Mom would be too.  
분류결과: 스포 X
다음 review는 spo:0.29659498207885304 <nonspo:0.703405017921147이므로 스포가 아니다.
2 번째 평점리뷰: 말이필요없다. 최고다. 끝날때까지 끝난게 아니다. 몰입도가 장난 아니다. 새로운 패러다임의 시작이다. 한국 배우여서 더 좋았다.  
분류결과: 스포 X
다음 review는 spo:1.8686418670618628e-48 <nonspo:6.815140697616737e-44이므로 스포가 아니다.
3 번째 평점리뷰: 근데 생각해보면 아빠 방구석에서 딸 SNS  뒤적이고 있을 때 마고 혼자 127시간 찍고있었음  
분류결과: 스포 O
다음 review는 spo:2.4588560140471663e-22 >nonspo:9.48558769126606e-23이므로 스포이다.
4 번째 평점리뷰: 이 영화가 감독의 첫 작품이라니 감독 천재인듯. 그리고 존조 연기 진짜 잘한다. 감독이 세번이나 찾아간 이유가 있는듯  
분류결과: 스포 X
다음 review는 spo:1.187488231039371e-59 <nonspo:6.728477976400175e-49이므로 스포가 아니다.
5 번째 평점리뷰: 엄청 노력한 게 티난다. 일반적인 영상 하나도 안 쓰려고 고민을 얼마나 했을까. 진짜 신선하다 완전 새롭고 세상 재밌다. 복선도 완벽하고. 광고 너무 해서 반감 샀었는데 광고 그만큼 할법하다. 이런 영화가 잊히면 얼마나 억울할까..  
분류결과: 스포 X
다음 review는 spo:4.1225047553768305e-97 <nonspo:5.236647899122728e-91이므로 스포가 아니다.
6 번째 평점리뷰: 내 더러운 상상력이 영화를 재밌게 한다...  
분류결과: 스포 X
다음 review는 spo:8.67197445773217e-15 <nonspo:1.913014858927779e-13이므로 스포가 아니다.
7 번째 평점리뷰: 마우스포인트로도 감정을 표현

34 번째 평점리뷰: 이거 스크린 좀 더 줬으면 좋겠음 진짜 반전에 반전 진짜 흥미 미진 진함 한국 스크린 독점 너무하네  
분류결과: 스포 X
다음 review는 spo:1.5414722472395841e-49 <nonspo:6.338585221160829e-44이므로 스포가 아니다.
35 번째 평점리뷰: 감독이 젊으니가 이런 신박한 타입의 영화를 낼 수있었는듯ㅋ 제발 흥행터져라  
분류결과: 스포 X
다음 review는 spo:1.0211504922870731e-43 <nonspo:5.954438310700058e-40이므로 스포가 아니다.
36 번째 평점리뷰: 영화 후반부에는 실시간으로 사건 시청하는 느낌이라 몰입감 장난아닙니다! 전자기기와 sns에 익숙한 세대들이라면 와- 하고 탄식하게 되는 부분도 많았습니다. 연출 특이하고 재밌었어용 강추  
분류결과: 스포 X
다음 review는 spo:7.392519457924411e-65 <nonspo:1.2665042585725338e-61이므로 스포가 아니다.
37 번째 평점리뷰: 식상하지 않은 스토리 반전있는 스토리였습니다 범인이 추측이 안가는!!  
분류결과: 스포 X
다음 review는 spo:1.3226120368957861e-24 <nonspo:1.4996093112661615e-23이므로 스포가 아니다.
38 번째 평점리뷰: 어쒸 방심했다가 개잼당했네ㅡㅡ;;  
분류결과: 스포 O
다음 review는 spo:9.617681033784168e-20 >nonspo:6.887955565030409e-20이므로 스포이다.
39 번째 평점리뷰: 영화에 대해 아무것도 안찾아보고 관람한게 신의 한수였다. 장르도 파격적인데 스토리도 훌륭하다.  
분류결과: 스포 X
다음 review는 spo:4.288245480469794e-37 <nonspo:3.5972774836361825e-35이므로 스포가 아니다.
40 번째 평점리뷰: 떡밥회수 완벽해서 놀람  
분류결과: 스포 X
다음 review는 spo:1.31393

67 번째 평점리뷰: 진짜  초딩때 반지의제왕 이후로 처음 댓글 다는 것 같은데 존잼. 와...연출력 지리고 스릴러 답게 반전에 반전 전혀 억지 없구요 진짜 소름 크게 한번 돋는 장면 있음. 유머포인트 지대로 살린 부분 많고 감동 있고 와 오천원 주고 본게 죄송해서  울뻔  
분류결과: 스포 X
다음 review는 spo:5.5488142150882113e-129 <nonspo:1.1822544912065593e-120이므로 스포가 아니다.
68 번째 평점리뷰: 맥북과 아이폰의 적절한 사용방법곧 삼성이 따라할듯  
분류결과: 스포 X
다음 review는 spo:7.988806409866115e-34 <nonspo:1.1617276951056967e-31이므로 스포가 아니다.
69 번째 평점리뷰: 와 미쳤다 이런 영화가  꿀잼이라니!!! 젭알 입소문 타라아아아 세상사람들 다 서치 보셔야됩니다!! 진심 알바아닙니다!  
분류결과: 스포 X
다음 review는 spo:2.039471453869085e-54 <nonspo:9.731668387157056e-52이므로 스포가 아니다.
70 번째 평점리뷰: 1시간반동안 모니터만 보다가 나왔다는 사실을 알아차렸을때, 너무나도 소름 돋음  
분류결과: 스포 X
다음 review는 spo:5.725593233315091e-27 <nonspo:4.267180967017533e-25이므로 스포가 아니다.
71 번째 평점리뷰: 가족을 지키느라 뛰는 아버지라는 것은 이런 걸 말하는 겁니다. ㅂㅅ같은 목X자 말고.  
분류결과: 스포 X
다음 review는 spo:1.3446098541090093e-29 <nonspo:1.6297142864055204e-28이므로 스포가 아니다.
72 번째 평점리뷰: 애플과 구글 광고보다 광고효과 좋은 영화 였음ㅎㅎㅎ맥프로 사고 싶음  
분류결과: 스포 X
다음 review는 spo:4.218089784409309e-33 <nonspo:1.7483967662966392e-31이므로 스포가 아니다.
73

100 번째 평점리뷰: 와 ㅅㅂ 이거 뭐냐 진짜 충격 그자체 너무 잘 만들었다. 그냥 보세요 무조건 입소문 타고 대박나자  
분류결과: 스포 X
다음 review는 spo:2.0724232447146556e-53 <nonspo:2.8891404231529766e-50이므로 스포가 아니다.
101 번째 평점리뷰: 처음 엄마의 얘기를그린 단편영화부터 메인스토리,실험적인 화면연출방식까지 뭐하나 나무랄데가 없는영화  
분류결과: 스포 X
다음 review는 spo:4.878479451137786e-51 <nonspo:1.6943110009076738e-46이므로 스포가 아니다.
102 번째 평점리뷰: 관람객폭우 뚫고 가서 보길 잘 했다.. 비 와서 더 이입 잘 됨!  
분류결과: 스포 O
다음 review는 spo:3.8470724135136674e-19 >nonspo:3.6735763013495515e-19이므로 스포이다.
103 번째 평점리뷰: 이거 진심 제작비 10억은 들었을까.. 진짜 잘 만들었다 깔끔하고 군더더기 없다. 다만 새로운 시도에만 유독 까칠해지는 신과 함께 천만 만드신 눈높은 관객에겐 비추.  
분류결과: 스포 X
다음 review는 spo:1.1284915663143466e-75 <nonspo:3.470319336232089e-73이므로 스포가 아니다.
104 번째 평점리뷰: 목격자보고 이거보니 진짜 한국영화는 많이 발전해야겠다는걸 느낌. 창의적인 시도와 전개.  개연성은 개나줘버리고 허구언날 경찰무능력만 찾아대는 목격와는 차원이 다른 클래스..  
분류결과: 스포 X
다음 review는 spo:4.662282941663733e-99 <nonspo:5.414754650298626e-96이므로 스포가 아니다.
105 번째 평점리뷰: 미니멀 라이프에 어울리는 트랜디한 영화.  맥을 사놓은게 후회되지 않는다  
분류결과: 스포 X
다음 review는 spo:1.757537410170545e-34 <nonspo:3.2782439368061984e-32이

133 번째 평점리뷰: 정말 잘 만든 영화. 영화 정말 좋은데 대박나면 좋겠어요ㅠㅠ  
분류결과: 스포 X
다음 review는 spo:8.728045538159133e-18 <nonspo:4.4128835319961507e-13이므로 스포가 아니다.
134 번째 평점리뷰: 잘만들었노? 레알로다  
분류결과: 스포 O
다음 review는 spo:4.8466482702293606e-12 >nonspo:1.2452531097333011e-12이므로 스포이다.
135 번째 평점리뷰: 진짜 개존잼 몰입도 최고고 올해 최고의 영화인듯  
분류결과: 스포 X
다음 review는 spo:1.0545224461023272e-33 <nonspo:8.605390334116273e-29이므로 스포가 아니다.
136 번째 평점리뷰: 와 미쳤다 이영화,별 기대없이 봤는데 꿀잼  
분류결과: 스포 X
다음 review는 spo:9.4472288349699e-26 <nonspo:2.4764889540726755e-23이므로 스포가 아니다.
137 번째 평점리뷰: 이런 영화가 1000만 찍어야 함  
분류결과: 스포 X
다음 review는 spo:1.421683492600612e-10 <nonspo:1.494303731679962e-08이므로 스포가 아니다.
138 번째 평점리뷰: 영화 진행측면에서 뒷통수를 1대 때리고, 내용에서 2대 더 맞고, 인물들 감정이 전해지는거 보고 1대 더 맞고... 총 4번 뒷통수맞은 영화  
분류결과: 스포 X
다음 review는 spo:8.294124831546913e-65 <nonspo:5.421878228459517e-60이므로 스포가 아니다.
139 번째 평점리뷰: 관람객<서치>는 반전에 반전을 거듭하는 미스터리 스릴러 영화다. 앞서 무심코 지나쳤던 이야기가 사건의 중요한 단서가 되고, 전혀 주목하지 않았던 인물이 유력한 용의자가 되는 등 관객을 쥐락펴락하는 솜씨가 탁월하고 치밀하다.  
분류결과: 스포 X
다음 review는 spo:8.7287719

...

854 번째 평점리뷰: 관람객독특한 영화였고, 재미있었어요  
분류결과: 스포 X
다음 review는 spo:4.335987228866085e-15 <nonspo:5.97817143414931e-13이므로 스포가 아니다.
855 번째 평점리뷰: 진짜로 신선하고 독특한 소재.. 정말 탄탄하다  
분류결과: 스포 X
다음 review는 spo:2.3466343866741903e-23 <nonspo:1.5991432123133493e-19이므로 스포가 아니다.
856 번째 평점리뷰: 정말 처음 봤던 촬영기법이자 새로운 영화였음.솔직히 스토리는 약간 음? 하는 부분이 있긴 했는데 그럼에도 배우들의 연기력과 반전과 독특한 촬영기법이 정말 좋았던 영화.  
분류결과: 스포 X
다음 review는 spo:6.972756904952428e-68 <nonspo:2.8678355841446805e-57이므로 스포가 아니다.
857 번째 평점리뷰: 새롭다 모든게 이런영화 최초  
분류결과: 스포 X
다음 review는 spo:2.9484713156289385e-13 <nonspo:2.0086656018741684e-12이므로 스포가 아니다.
858 번째 평점리뷰: 극장에서 박수칠 뻔 했다  
분류결과: 스포 X
다음 review는 spo:1.051148419119051e-15 <nonspo:5.021664004685419e-15이므로 스포가 아니다.
859 번째 평점리뷰: 평 보고 지금 막 보고 나왔는데..저는 그닥  하는일도 컴 관련일인데..피OO  다녀온 느낌? 머리가 띵하네요 다시 보고 싶지 않은데 모가 재밋다는건지  
분류결과: 스포 X
다음 review는 spo:3.7628217831971183e-65 <nonspo:1.948460095149955e-63이므로 스포가 아니다.
860 번째 평점리뷰: 참 신박한 영화다 단 한장면도 오프라인 장면 없이 온라인으로 연결된 장면으로 영화를 구성한 아이디어에 점수를 준다. 반전을 가미한 치밀한 스토리와 디텔일한 소셜미디어 묘사는 

888 번째 평점리뷰: 무심하게 지나치던 것들. 그리고 무심하게 지나칠 수 있었던 것들.  
분류결과: 스포 X
다음 review는 spo:8.799878950028213e-24 <nonspo:1.410879040364686e-22이므로 스포가 아니다.
889 번째 평점리뷰: 독특한 촬영 방식, 충격적인 스토리 전개...이영화는 영화 역사를 바꿀 환상적인 영화입니다. 촬영방식이 너무 마음에드네요ㅎㅎ  
분류결과: 스포 X
다음 review는 spo:2.9749395813006058e-59 <nonspo:2.2515538877288364e-52이므로 스포가 아니다.
890 번째 평점리뷰: 이 영화는 컴퓨터나 폰으로 볼 수가 없다. 상영할때 꼭 영화관에서 봐야하는 영화임.  
분류결과: 스포 X
다음 review는 spo:9.092957753658528e-25 <nonspo:1.8668916730701715e-20이므로 스포가 아니다.
891 번째 평점리뷰: 관람객평점 보고 영화 보러 갔는데 그냥 그저 그랬네요 ~ 반전 이라기 보단 ....  
분류결과: 스포 X
다음 review는 spo:2.8817009628757016e-35 <nonspo:2.5900397882180516e-33이므로 스포가 아니다.
892 번째 평점리뷰: 노트북화면으로만 만들려고 되도않는 영상통화억지오짐. 그리고 비번은찾으면서 왜 문자는 처음부터 안OO? 미국형사시스템이 무슨 저렇게 허술할걱같진않음. 그리고 구글링에뜬사진이 우연히도 같이찍은사진일 확률은?ㅋㅋ  
분류결과: 스포 X
다음 review는 spo:1.2926063066107505e-91 <nonspo:8.238706620463614e-89이므로 스포가 아니다.
893 번째 평점리뷰: 반전에 또다른반전 ..집중하느라 팝콘 콜라 다 남김..  이영화는 상영관 많아라!!  
분류결과: 스포 X
다음 review는 spo:1.4710693991887577e-40 <nonspo:1.0529954065237995e-37이므로 스포가 아니다

922 번째 평점리뷰: 좀 슬픈 느낌이 깔려있지만 재밌네요~ 반전도 좋고~  
분류결과: 스포 X
다음 review는 spo:1.7953004596397115e-18 <nonspo:2.644974936971677e-17이므로 스포가 아니다.
923 번째 평점리뷰: 진짜 존잼 꼭보셈 돈이 안아까움  
분류결과: 스포 X
다음 review는 spo:3.578495770821932e-26 <nonspo:1.493513338456137e-24이므로 스포가 아니다.
924 번째 평점리뷰: 최소한의 정보를 가지고 이 영화 보시면 더 재미있습니다. 진짜 반전과 복선이 최고예요.  강추합니다  
분류결과: 스포 X
다음 review는 spo:1.0005906121096184e-37 <nonspo:5.665712036726987e-32이므로 스포가 아니다.
925 번째 평점리뷰: 와 존잼임진짜 영화보면서 처음으로 감성펑달음  
분류결과: 스포 X
다음 review는 spo:2.460552374238763e-33 <nonspo:5.327146397310073e-31이므로 스포가 아니다.
926 번째 평점리뷰: 꼭 봐.... 레알루다가 인생 띵작!!  
분류결과: 스포 X
다음 review는 spo:5.866585966685476e-24 <nonspo:3.0862979007977515e-23이므로 스포가 아니다.
927 번째 평점리뷰: 스릴러는 내용 전체가 스포라 뭐라 적을까 망설이다가 관뒀다. 그냥 보라는 말 밖에는 할 수 없다. 아마 당신이 예상하는 전개는 아닐 것이기에  
분류결과: 스포 X
다음 review는 spo:3.108634867071984e-51 <nonspo:1.123297796521878e-47이므로 스포가 아니다.
928 번째 평점리뷰: 연출방법도 독특하고 스토리도 탄탄한것 같음. 한국형 신파극과 클리셰에 질려있는참에 신선한영화라 좋았음.  
분류결과: 스포 X
다음 review는 spo:2.297588607645914e-44 <nonspo:1.0993574

956 번째 평점리뷰: 너의결혼식같은 쓰레기 영화보지말고 서치를 봐라  
분류결과: 스포 X
다음 review는 spo:7.098569019689425e-22 <nonspo:1.6313288904216683e-20이므로 스포가 아니다.
957 번째 평점리뷰: 졸리다 재미없다는애들 죄다 급식일듯 ㅋㅋ  
분류결과: 스포 X
다음 review는 spo:8.799878950028213e-24 <nonspo:3.306747750854733e-23이므로 스포가 아니다.
958 번째 평점리뷰: 이거 보고나서 사람 거를 수 있게 됨.... 이 영화 평점테러하는 사람 차단각;  
분류결과: 스포 X
다음 review는 spo:7.87269069580825e-27 <nonspo:8.466628902812566e-26이므로 스포가 아니다.
959 번째 평점리뷰: 흠..별점이 너무높아서 너무기대했나봄ㅠ  
분류결과: 스포 X
다음 review는 spo:2.815961264009028e-22 <nonspo:4.650114024639468e-20이므로 스포가 아니다.
960 번째 평점리뷰: 노잼 ㅜㅜ 내용 기억도 안나ㅜㅜ  
분류결과: 스포 X
다음 review는 spo:1.9235362067568337e-19 <nonspo:2.5829833368864036e-19이므로 스포가 아니다.
961 번째 평점리뷰: 머리털나고 첨 평점 등록하는데...  이건 무조건 10점... 쇼생크 탈출도 9.5인데... 이 작품은 완벽하다.... 결말에 엔딩 크레딧까지...  
분류결과: 스포 O
다음 review는 spo:2.1491796611855686e-53 >nonspo:1.9967934802876078e-53이므로 스포이다.
962 번째 평점리뷰: 보는 내내 집중할 수 밖에 없는 영화! 끝까지 소망을 놓지 않는 아버지의 사랑.  
분류결과: 스포 X
다음 review는 spo:2.4247887343089413e-24 <nonspo:9.412245518395448e-22이므로 스포가 아니다.
9

991 번째 평점리뷰: 관람객완전별로  완전별로  완전별로  
분류결과: 스포 X
다음 review는 spo:9.542655388858485e-28 <nonspo:2.709321248900021e-26이므로 스포가 아니다.
992 번째 평점리뷰: 영화왠만한거 다 불평없이 보는데 영화가너무졸려요ㅠㅠ 너무 너무너무 지루해요ㅠㅠ 평점좋아서봤는데 정말실망이에요 17년도-OOO도 현재 까지 본 영화중에서 가장 실망했어요..  
분류결과: 스포 X
다음 review는 spo:1.2220694623920135e-73 <nonspo:2.775795976642638e-65이므로 스포가 아니다.
993 번째 평점리뷰: 솔직히 앞부분 ㅈㄴ 재미없었다 ㅇㅈ?중간부터 재밌어지긴 했는데 그래도 노잼이고 지루했었음똑같은 화면 구성만 계속 보니까 빡치드라그나마 뒷부분 땜에 4점 줌  
분류결과: 스포 X
다음 review는 spo:7.687068826919793e-82 <nonspo:1.200508126848145e-79이므로 스포가 아니다.
994 번째 평점리뷰: 관람객왜 1위인지 모르겠다.그냥 쏘쏘...  
분류결과: 스포 X
다음 review는 spo:9.617681033784168e-20 <nonspo:1.3948110019186577e-18이므로 스포가 아니다.
995 번째 평점리뷰: 생각보다는 재미없었다.  
분류결과: 스포 X
다음 review는 spo:3.618335757946237e-05 <nonspo:0.00013507537550094036이므로 스포가 아니다.
996 번째 평점리뷰: 정말 시간이 너무 빨리 지나갔다.최고다!!  
분류결과: 스포 X
다음 review는 spo:3.519951580011285e-23 <nonspo:3.0752754082949027e-19이므로 스포가 아니다.
997 번째 평점리뷰: pc방온줄알았네 종일 키보드와 마우스  
분류결과: 스포 X
다음 review는 spo:4.808840516892084e-20 <nonspo:2.75518222

# 영화 해피데스데이

In [10]:
file_name = "해피데스데이_최종본.csv"
uptitle= "해피데스데이_상위tf.csv"
ctitle = "해피데스데이_네이버평점.csv" 

csv_file = open(file_name, 'r')
doc = csv.reader(csv_file)
next(doc) # header 넘어가기
    
# 문장별
sentences = []
for line in doc:
    sentences.extend(split_sentence(line))
# 형태소별    
sentence_morphs = []
for s in sentences:
        sentence_morphs.append(preprocessing(s))    

for a in sentence_morphs:
    if a == []:
        sentence_morphs.remove(a)    

# 상위tf 2차 전처리   
f=open(uptitle,'r')
rf=csv.reader(f)
next(rf)
TF=[]
for row in rf:
    TF.append(row[0])
stop=['이', '을', '스', '여','노', '은', '아', '터', '드', '마', '자', '고', '레','거','개방','관람','씬','내용','결국','그런데',
      '거','늘', '의', '라', '누', '어', '건','진','자', '마','로','스','까','은','시','드','곽','왜','그리하','약간','갑자기',
      '마','저','거','라','까','분','늘','자','의','어','스','여','을','몇','몇몇','걸','시','드','단','서','인','블록버스터',
      '늘','메','파','자','김','티','리','바','스','건','대','새','어','보','시','디','르','을','고','예고편','스포일러','장르',
       '어쩌','듯하','경','어느','결말','이때','재미있','그리하','이번','조금','스','자','누','터','라','뜨','다','막','분','임',
      '위','하다','리','사','유','면','감독','스포','줄거리','영화']
newTF=[]
for word in TF:
    if word not in stop:
        newTF.append(word)
    newTF=[]
    for word in TF:
        if word not in stop:
            newTF.append(word)   

dataset2=[]
for i in range(len(sentences)):
    a=split_morphs(sentences[i])
    dataset= []
    for w in a:
        if w in newTF: 
            dataset.append(w)
    dataset2.append(dataset)
    
dataset3=[]
for k in range(len(dataset2)):
    if dataset2[k] != []:
        dataset3.append(dataset2[k])
 ############################ Apriori 알고리즘 적용 ################################
te = TransactionEncoder()
te_ary = te.fit(dataset3).transform(dataset3)
df = pd.DataFrame(te_ary, columns=te.columns_) #위에서 나온걸 보기 좋게 데이터프레임으로 변경
    
frequent_itemsets3 = apriori(df, min_support=0.003, max_len=3, use_colnames=True)
frequent_itemsets2 = apriori(df, min_support=0.003, max_len=2, use_colnames=True)

frequentThree=frequent_itemsets3[len(frequent_itemsets2):len(frequent_itemsets3)]
frequentThree=frequentThree.sort_values("support",ascending=False)

# 일정 확률 이상의 데이터 선택
frequentThree=frequentThree[frequentThree["support"]>=0.011]

# frozenset를 list로 변화
sets=[]
for line in frequentThree['itemsets']:
    sets.append(line)
association=[list(x) for x in sets]

In [11]:
############################### 나이브 베이지안 알고리즘 적용 ################################
spo = []
nonspo = []

for row in sentence_morphs:
    s1 = set(row)
    for dataset in association:
        s2 = set(dataset)
        isSpo = False
        if s2.issubset(s1) == True:
            spo.append(row)
            isSpo = True
            break
    if not isSpo:
        nonspo.append(row)
        
csv_file = open(ctitle, 'r')
doc = csv.reader(csv_file)
next(doc) # header 넘어가기

review_token = []
totalreview = []
for line in doc:
    text = line[2]
    totalreview.append(text) # totalreview 리스트에 결과 저장
    
index = 0
for comment in totalreview[0:1000]:
    review=preprocessing(comment) # 형태소 분석
    index+=1
    
    
    #__________prior probabilities
    prior_spo=len(spo)/(len(spo)+len(nonspo))
    prior_nonspo=len(nonspo)/(len(spo)+len(nonspo))
    
    #_________training data
    
    #spo의 중복단어 리스트 (the sum of term frequency for class SPO)
    spo_overlap=[]
    for i in range(len(spo)):
        spo_overlap.extend(spo[i])
        
    #nonspo의 중복단어 리스트(the sum of term frequency for class NONSPO)
    nonspo_overlap=[]   
    for i in range(len(nonspo)):
        nonspo_overlap.extend(nonspo[i])
    
    #중복없는 전체단어 갯수: Vocabulary
    all_spo=[]
    for i in range(len(spo)):
        for j in range(len(spo[i])):
            if(spo[i][j] not in all_spo):
                all_spo.append(spo[i][j])
                
    all_nonspo=[]
    for i in range(len(nonspo)):
        for j in range(len(nonspo[i])):
            if(nonspo[i][j] not in all_nonspo):
                all_nonspo.append(nonspo[i][j])
    
    overall=[]
    overall.extend(all_spo)
    overall.extend(all_nonspo)
    
    nonover=[]
    for i in range(len(overall)):
        if(overall[i] not in nonover):
            nonover.append(overall[i])
    all_count=len(nonover)
    
    #_________likelihood
    
    #spo의  likelihood
    spo_like_result=1
    for i in range(len(review)):
        count=1# Laplace smoothing
        for j in range(len(spo_overlap)):
            if (review[i]==spo_overlap[j]):
                count=count+1
        spo_like_result=spo_like_result*(count/(len(spo_overlap)+all_count))#review 단어별 like
        
    
    #nonspo의  likelihood
    nonspo_like_result=1
    for i in range(len(review)):
        ncount=1# Laplace smoothing
        for j in range(len(nonspo_overlap)):
            if (review[i]==nonspo_overlap[j]):
                ncount=ncount+1
        nonspo_like_result=nonspo_like_result*(ncount/(len(nonspo_overlap)+all_count))#review 단어별 like 
        
    #_________posterior probabilities
    
    #P(SPO | REVIEW)=p(REVIEW|SPO)P(SPO)
    post_spo=spo_like_result*prior_spo
    
    #P(NONSPO | REVIEW)=P(REVIEW|NONSPO)P(NONSPO)
    post_nonspo=nonspo_like_result*prior_nonspo
    if(post_spo>post_nonspo):
        print(index,"번째 평점리뷰:",comment)
        print("분류결과: 스포 O")
        print("다음 review는 spo:{} >nonspo:{}이므로 스포이다.".format(post_spo,post_nonspo))
        print("=============================================================================================")
    
    elif(post_spo<post_nonspo):
        print(index,"번째 평점리뷰:",comment)
        print("분류결과: 스포 X")
        print("다음 review는 spo:{} <nonspo:{}이므로 스포가 아니다.".format(post_spo,post_nonspo))
        print("=============================================================================================")

        

1 번째 평점리뷰: 팝콘만 사고 음료수는 사지 마세요. 여주가 핵사이다  
분류결과: 스포 X
다음 review는 spo:1.2343268443658916e-30 <nonspo:2.438646814793656e-28이므로 스포가 아니다.
2 번째 평점리뷰: 관람객이건 완전 새로운 공포영화 ㅋㅋㅋ 무섭고 유쾌하고 혼자 다 하셨다  
분류결과: 스포 X
다음 review는 spo:7.7652324893702104e-31 <nonspo:7.409128006380726e-28이므로 스포가 아니다.
3 번째 평점리뷰: 관람객무서울줄알았는데 뭔가 유쾌한 공포?느낌이예요 저는 진짜재밋던데...?  
분류결과: 스포 X
다음 review는 spo:4.782800139672373e-40 <nonspo:1.7302792245443324e-37이므로 스포가 아니다.
4 번째 평점리뷰: 관람객남녀가 맨날 보면 정드는거 맞구나  
분류결과: 스포 X
다음 review는 spo:1.705180621454825e-21 <nonspo:2.4685655274300097e-21이므로 스포가 아니다.
5 번째 평점리뷰: 이 영화 보기전으로 리셋하고 싶다. 다시 보고 싶네요  
분류결과: 스포 X
다음 review는 spo:1.3777749311617723e-06 <nonspo:2.589180756109025e-06이므로 스포가 아니다.
6 번째 평점리뷰: 겟 아웃 제작진이 ㅂㅅ미를 장착할 때  
분류결과: 스포 X
다음 review는 spo:4.414569958874611e-27 <nonspo:5.6437254856653156e-27이므로 스포가 아니다.
7 번째 평점리뷰: 관람객공포영환데 보고나면 행복해지고 기분 개죠아짐ㅋ 이거 거의 뭐 영화소재 신선도100퍼센트에 몰입도, 연출, 배우연기까지 쌍타취~ 개추천함 꼬보셈ㅋ  
분류결과: 스포 X
다음 review는 spo:1.1505093216176961e-94 <nonspo:1.170824470950696e-87이므로 스포가 아니다.
8 번째

37 번째 평점리뷰: 미국에서 봤는데  스릴있고 재미있어요.  
분류결과: 스포 X
다음 review는 spo:3.8522995419034594e-12 <nonspo:2.3503819499900373e-10이므로 스포가 아니다.
38 번째 평점리뷰: 공포스럽고 재미있다여주 매력 포텐 터지네요ㅋㅋ사랑의 블랙홀 공포영화버전? ㅋㅋ2탄 나와도 괴안을듯 하네요~  
분류결과: 스포 X
다음 review는 spo:7.214406776662576e-52 <nonspo:8.880070525066795e-47이므로 스포가 아니다.
39 번째 평점리뷰: 여주인공이 볼수록 매력있네요 영화도 그럭저럭 오락용...  
분류결과: 스포 X
다음 review는 spo:6.023287400131537e-18 <nonspo:2.610596913616223e-16이므로 스포가 아니다.
40 번째 평점리뷰: 친구랑 정말 재밌게 본 영화. 둘다 쫄보여서 서로 팔 붙잡고 봤는 데 정말 재밌었음!! 공포영화 못보는 사람(=나)도 볼 만할 듯! 조금 아쉬운 건 개연성이 탄탄하진 않다는 점? 가볍게 보면 정말 좋다  
분류결과: 스포 X
다음 review는 spo:1.0095719297195285e-89 <nonspo:1.1232675932952389e-80이므로 스포가 아니다.
41 번째 평점리뷰: 반전이 상당하네요 제대로 볼만했던 영화예요  
분류결과: 스포 X
다음 review는 spo:7.52910925016442e-18 <nonspo:7.179141512444613e-16이므로 스포가 아니다.
42 번째 평점리뷰: 관람객겟아웃도 꿀잼이었는데... 해피데스데이도 꿀잼이다 영상도 독특하고 예고편이 다라고 생각하고 봤는데 반전있음.... 강추다  
분류결과: 스포 X
다음 review는 spo:4.723718722814783e-57 <nonspo:2.905829931944578e-52이므로 스포가 아니다.
43 번째 평점리뷰: 관람객쫄보도 웃으면서 볼수있는공포영화  
분류결과: 스포 X
다음 re

71 번째 평점리뷰: 재밋엇음. 이정도면 딱 좋은듯  
분류결과: 스포 X
다음 review는 spo:7.357616598124351e-27 <nonspo:3.3862352913991904e-26이므로 스포가 아니다.
72 번째 평점리뷰: 그냥 무난히 볼만한 7점정도 영화인데 너무 고평가 되었음  
분류결과: 스포 X
다음 review는 spo:1.8946451349498048e-21 <nonspo:4.834438728918933e-19이므로 스포가 아니다.
73 번째 평점리뷰: 무섭지도 않고 재밌지도 않고 개연성도 없고! 반전이라 할 수 없는 반전에 중간에 감동유도는 굉장히 난데없어서 한국영화 보는 줄. 짧은 러닝타임인데도 지루했고 기억에 남는 건 여주분 연기 뿐이니 영화비가 아까워 짜증날 정도. 지금은 너무 고평가된 평점임  
분류결과: 스포 X
다음 review는 spo:3.573607626541984e-107 <nonspo:5.624333188660519e-97이므로 스포가 아니다.
74 번째 평점리뷰: 평점 장난치나 진심 재미없음 ㅡㅡ 계속 죽고 살고 죽고 살고핵노잼 진짜 최악의 영화 웬만해서 평점 안 다는데 돈 아까워서 달았음  
분류결과: 스포 X
다음 review는 spo:7.047368457050117e-65 <nonspo:3.092219751893195e-62이므로 스포가 아니다.
75 번째 평점리뷰: 공포영화인척 하는 코미디 삼류영화. 연출부터 음악까지 구닥다리 호러무비의 진행법을 따르는듯. 억지로 쑤셔넣은 자극적이고 극단적인 유머식은 유치해서 고개를 못 들었다. 반전은 커녕 그냥 막장전개에 어이가 없었음.  
분류결과: 스포 X
다음 review는 spo:7.083114224877481e-100 <nonspo:1.3973186748921489e-93이므로 스포가 아니다.
76 번째 평점리뷰: 평점보고 봤는데.진짜 재미 없습니다. 저는 진실 된 사람입니다. 저를 믿고 보지 마세요.  
분류결과: 스포 X
다음 review는 spo:1.932

103 번째 평점리뷰: 오늘 보고왔는데 겟아웃보다 훨씬 재밌고 볼 만했어요~ 조금씩 웃긴부분도있구요  
분류결과: 스포 X
다음 review는 spo:2.4686536887317832e-29 <nonspo:1.8289851110952418e-25이므로 스포가 아니다.
104 번째 평점리뷰: 소재도 신선하고 가볍게 볼만하네요.  주인공도 매력있어요.  
분류결과: 스포 X
다음 review는 spo:2.4209354502136402e-21 <nonspo:9.031658409690598e-19이므로 스포가 아니다.
105 번째 평점리뷰: 1점도 아까운데..공포영환줄알고 기대하고 봤다가 시간 낭비 돈낭비  
분류결과: 스포 X
다음 review는 spo:4.8892204562701326e-33 <nonspo:3.584136999990674e-31이므로 스포가 아니다.
106 번째 평점리뷰: 저는 그다지 재미가 없었네요 저번주에 토르를 봐서 그런가 저예산 영화같고 스토리도이상하게 흘러가고  
분류결과: 스포 X
다음 review는 spo:1.6404486362432994e-35 <nonspo:2.321686061569938e-33이므로 스포가 아니다.
107 번째 평점리뷰: 이거 재밌다는 놈들 혼자 보기 억울한거다 이건 진짜다.  
분류결과: 스포 X
다음 review는 spo:3.6104060197702327e-29 <nonspo:2.2862313888690525e-27이므로 스포가 아니다.
108 번째 평점리뷰: 별 한개도 아까움... 왜 이런일이 생겼는지 이유도 뭐도 없고 범인 반전도 이상하고 자꾸 똑같은 장면만나와서 지루함.  
분류결과: 스포 O
다음 review는 spo:4.0685107379361706e-46 >nonspo:1.6734301101494497e-46이므로 스포이다.
109 번째 평점리뷰: 관람객나름 기대하고 봤는데 기대치도 못하고 실망했어요  
분류결과: 스포 X
다음 review는 spo:1.5058218500328843e-19 <nons

137 번째 평점리뷰: 개 노 잼 ㅡㅡ 돈아까움 근데 그래서 왜 꿈을 많이 꾸게돴는지도 안나오고차라리 카터가 범인이면 재밌기라도하지  
분류결과: 스포 X
다음 review는 spo:4.4828960908741115e-46 <nonspo:3.904670257015385e-45이므로 스포가 아니다.
138 번째 평점리뷰: 초반에 총 준비했다가 쏴버리면 바로 해결되는데 왜 계속 고생인지?  
분류결과: 스포 X
다음 review는 spo:9.645837981110595e-33 <nonspo:9.46428494510569e-32이므로 스포가 아니다.
139 번째 평점리뷰: 관람객이게 뭔가요...아 개노잼  
분류결과: 스포 X
다음 review는 spo:4.7677355555845785e-25 <nonspo:1.0158705874197574e-24이므로 스포가 아니다.
140 번째 평점리뷰: 진짜 재미없고 돈 아깝고 이정도면 평점 조작인거 같다 8점대는 무슨 1점이 충분하다  
분류결과: 스포 X
다음 review는 spo:1.1837844082725427e-39 <nonspo:4.610283460134569e-39이므로 스포가 아니다.
141 번째 평점리뷰: 엥간. 배우들이 매력적이었음. 그리고는 뭐... 약간 학창시절 기말고사 끝나고 시간 남을때 선생님이 틀어줄 것 같은 영화임. 굳이 시간내서 영화관 가서 봐야할까.  
분류결과: 스포 X
다음 review는 spo:7.26912271164509e-63 <nonspo:5.292466629068852e-60이므로 스포가 아니다.
142 번째 평점리뷰: 영화를 볼 때 제일 중요한것이 장르를 불문하고 개연성이 중요하다고 생각함 즉 등장인물들의 행동,생각 등등이 설득력이 있어야한다는것인데, 이 영화는 그 개연성이 라는 것이 심각하게 붕괴되어있음. 돈이 아까운 영화까진 아니지만, 극장에서 보긴 아깝다.  
분류결과: 스포 X
다음 review는 spo:3.8750831176406865e-84 <nonspo:2.254757

...

858 번째 평점리뷰: 관람객재미쎵재미쎵 반전도있자넝|~  
분류결과: 스포 X
다음 review는 spo:7.357616598124351e-28 <nonspo:7.760122542789811e-25이므로 스포가 아니다.
859 번째 평점리뷰: 무서운데 병맛이고 재미있는데 병맛 꾸르잼  
분류결과: 스포 X
다음 review는 spo:1.8514902665488375e-30 <nonspo:2.9418596495923455e-28이므로 스포가 아니다.
860 번째 평점리뷰: 관람객진짜 존잼 꿀잼 10자자자  
분류결과: 스포 X
다음 review는 spo:2.674374829459431e-29 <nonspo:2.438646814793655e-28이므로 스포가 아니다.
861 번째 평점리뷰: 공포영화보면서  설레이고 행복하게 느껴진다는게 참 심기하다  시간가는줄 모르고 봤는데  여주인공  매력쩜 ~  
분류결과: 스포 X
다음 review는 spo:5.3787233555531223e-45 <nonspo:1.3652068512602644e-41이므로 스포가 아니다.
862 번째 평점리뷰: 쫄보인 나도 재밌었음. 참고로 범죄도시는 반 못 봤음. 근데 이건 다 봄.  
분류결과: 스포 X
다음 review는 spo:3.394765426082778e-44 <nonspo:7.046876541064009e-41이므로 스포가 아니다.
863 번째 평점리뷰: 타임슬립, 캠퍼스호러, 반전영화의 절묘한 조합♡ 호러씬은 오랜만에 스크림 떠올릴정도로 장르적 연출이 뛰어났고 타임슬립씬은 전혀 지루하지않았고 반전도 훌륭하다. 특히 ost 가 뛰어난 것도 맘에 든다. 범죄도시에 이어 2연타로 영화초이스 성공이당  
분류결과: 스포 X
다음 review는 spo:5.342578554302407e-135 <nonspo:1.9749952603681723e-128이므로 스포가 아니다.
864 번째 평점리뷰: 관람객유쾌한 공포에  끝이 깔끔한 영화!!  
분류결과: 스포 X
다음 review는 sp

892 번째 평점리뷰: 무난무난하니 나쁘지않았던 스릴무비였습니당  
분류결과: 스포 X
다음 review는 spo:2.258732775049326e-19 <nonspo:6.046603145508164e-18이므로 스포가 아니다.
893 번째 평점리뷰: 진짜 평점남기는거 처음인 영화 꼭. 보셈 예고편보고 개봉하기기다리면서 기대 엄청하고 봤는데도 개꿀잼 인생영화. ㅠㅠ이런재밌는거 안본사람 없었으면  
분류결과: 스포 X
다음 review는 spo:3.669935166072143e-74 <nonspo:1.0685313013600097e-67이므로 스포가 아니다.
894 번째 평점리뷰: 관람객그럭저럭 한 영화이면서도, 뭔가 나중에 교훈을 주네요. 있을때 잘하란 말이죠. 젤 가까운 사람이 좋으면서도 무서운 영화였어요. 반전에 반전이 있네요? 하 ㅋㅋ 남친이랑 심장 쫄면서 본영화.. ㅋㅋㅋㅋ  
분류결과: 스포 X
다음 review는 spo:1.8247901022001624e-62 <nonspo:1.1803210222611244e-56이므로 스포가 아니다.
895 번째 평점리뷰: 오늘을 열심히..! ㅋㅋㅋ내용도있고 재밌었음  
분류결과: 스포 X
다음 review는 spo:4.847014661978349e-15 <nonspo:7.556526330986488e-13이므로 스포가 아니다.
896 번째 평점리뷰: 마음울적했는데 이거보고 좀 나아졌어요 ♡  
분류결과: 스포 X
다음 review는 spo:6.731964808303263e-16 <nonspo:3.3584561471051057e-15이므로 스포가 아니다.
897 번째 평점리뷰: 관람객존잼 ㅠㅜ무서운데웃기고 남주 귀엽고  
분류결과: 스포 X
다음 review는 spo:5.827232345714486e-25 <nonspo:3.9032005458861335e-23이므로 스포가 아니다.
898 번째 평점리뷰: 관람객급하게 상영시간 맞춰서 보고 왔습니다. 예고를 하도 봐서 재미없으면 어떡하나 했지만 다행이도 예고만큼 스토

927 번째 평점리뷰: 난 잼있게봄 내일이 온다는거에 감사를  스릴런데 훈훈하노 ㅋㅋ  
분류결과: 스포 X
다음 review는 spo:3.5645036973869155e-43 <nonspo:1.129307137991027e-42이므로 스포가 아니다.
928 번째 평점리뷰: 엄청난 충격과 공포를 준 나의 처음이자 마지막이될 영화일것이다.  
분류결과: 스포 X
다음 review는 spo:3.240107966460465e-27 <nonspo:2.7795928647095682e-24이므로 스포가 아니다.
929 번째 평점리뷰: 이렇게 재밌는 영화 가능한거였습니까?  
분류결과: 스포 X
다음 review는 spo:8.131437990177575e-17 <nonspo:1.7621529166909507e-14이므로 스포가 아니다.
930 번째 평점리뷰: 겟아웃보다 더예상치못한 스토리와 반전 짱임ㅋㅋㅋ  
분류결과: 스포 X
다음 review는 spo:2.258732775049326e-19 <nonspo:1.90036098858828e-16이므로 스포가 아니다.
931 번째 평점리뷰: 와..지루하지않으면서 반전오졌다리~  
분류결과: 스포 X
다음 review는 spo:3.2313431079855664e-15 <nonspo:2.2165810570893705e-13이므로 스포가 아니다.
932 번째 평점리뷰: 여주캐릭터가 한 매력풍긴다ㅋㅋ  
분류결과: 스포 X
다음 review는 spo:3.2313431079855664e-15 <nonspo:6.269118141262864e-14이므로 스포가 아니다.
933 번째 평점리뷰: 영화고민 엄청 했는데 후회없다 돈더주고봐도될 가치있음  
분류결과: 스포 X
다음 review는 spo:2.943046639249741e-25 <nonspo:6.140373328403864e-24이므로 스포가 아니다.
934 번째 평점리뷰: 여주매력넘치고 이쁘네...반만닮았으면..  
분류결과: 스포 X
다음 review는 spo:1.26309675

962 번째 평점리뷰: 강추 개꿀잼 이 영화로 여자배우 팬 됨ㅋㅋ그럼 이제 이 배우 이름을 알아봐야 겠군  
분류결과: 스포 X
다음 review는 spo:4.82484893012735e-36 <nonspo:4.952930264682535e-34이므로 스포가 아니다.
963 번째 평점리뷰: 내가원하던 스타일이다 코믹하면서도 공포감조성  
분류결과: 스포 X
다음 review는 spo:7.529109250164422e-20 <nonspo:2.30346786495549e-18이므로 스포가 아니다.
964 번째 평점리뷰: 지루하지 않고 잼있음.  
분류결과: 스포 X
다음 review는 spo:1.1556898625710377e-11 <nonspo:2.61153549998893e-11이므로 스포가 아니다.
965 번째 평점리뷰: 여러장르가 고루베여있는 느낌을 받았어요  
분류결과: 스포 X
다음 review는 spo:6.776198325147978e-19 <nonspo:5.758669662388726e-18이므로 스포가 아니다.
966 번째 평점리뷰: 솔직히 겟아웃보단 재밌는듯 이해도잘되고 웃김  
분류결과: 스포 X
다음 review는 spo:2.9430466392497406e-26 <nonspo:7.999980875930588e-24이므로 스포가 아니다.
967 번째 평점리뷰: 관람객친구랑 보러갔는데, 무섭기만 할 줄 알았는데 엄청 웃고 왔어요. ㅋㅋㅋ새로운 장르임 ㅋㅋㅋ 근데 재밌어요. 추천추천합니다.  
분류결과: 스포 X
다음 review는 spo:9.439659488995729e-44 <nonspo:2.6701839884943397e-41이므로 스포가 아니다.
968 번째 평점리뷰: 관람객개꿀잼 ㅋㅋㅋ반전 나는 전혀 예상 못함  
분류결과: 스포 X
다음 review는 spo:1.0525806305276697e-23 <nonspo:1.8102813867820068e-21이므로 스포가 아니다.
969 번째 평점리뷰: 저는 무난하게 재밌게봤습니다~ 여주가 너

997 번째 평점리뷰: 아무 생각없이 보면 더 재밌음ㅋㅋ마지막에 메시지도 있고  
분류결과: 스포 X
다음 review는 spo:4.743338827603585e-18 <nonspo:2.8793348311943635e-16이므로 스포가 아니다.
998 번째 평점리뷰: 보고 팬티 세번 갈아입음  
분류결과: 스포 X
다음 review는 spo:3.8522995419034594e-12 <nonspo:1.305767749994465e-11이므로 스포가 아니다.
999 번째 평점리뷰: 무서운분위기없이 걍 존잼임ㅋㅋ반전도 쩔고  
분류결과: 스포 X
다음 review는 spo:4.969748793196935e-32 <nonspo:3.329265035546894e-30이므로 스포가 아니다.
1000 번째 평점리뷰: 무섭지만 무섭지않은 영화공포와 코믹과 감동이 잘 어우러진 영화  
분류결과: 스포 X
다음 review는 spo:1.3330729919151633e-25 <nonspo:9.30456076948507e-22이므로 스포가 아니다.


# 영화 곡성

In [12]:
file_name = "곡성_최종본.csv"
uptitle= "곡성_상위tf.csv"
ctitle = "곡성_네이버평점.csv" 

csv_file = open(file_name, 'r')
doc = csv.reader(csv_file)
next(doc) # header 넘어가기
    
# 문장별
sentences = []
for line in doc:
    sentences.extend(split_sentence(line))
# 형태소별    
sentence_morphs = []
for s in sentences:
        sentence_morphs.append(preprocessing(s))    

for a in sentence_morphs:
    if a == []:
        sentence_morphs.remove(a)    

# 상위tf 2차 전처리   
f=open(uptitle,'r')
rf=csv.reader(f)
next(rf)
TF=[]
for row in rf:
    TF.append(row[0])
stop=['이', '을', '스', '여','노', '은', '아', '터', '드', '마', '자', '고', '레','거','개방','관람','씬','내용','결국','그런데',
      '거','늘', '의', '라', '누', '어', '건','진','자', '마','로','스','까','은','시','드','곽','왜','그리하','약간','갑자기',
      '마','저','거','라','까','분','늘','자','의','어','스','여','을','몇','몇몇','걸','시','드','단','서','인','블록버스터',
      '늘','메','파','자','김','티','리','바','스','건','대','새','어','보','시','디','르','을','고','예고편','스포일러','장르',
       '어쩌','듯하','경','어느','결말','이때','재미있','그리하','이번','조금','스','자','누','터','라','뜨','다','막','분','임',
      '위','하다','리','사','유','면','감독','스포','줄거리','영화']
newTF=[]
for word in TF:
    if word not in stop:
        newTF.append(word)
    newTF=[]
    for word in TF:
        if word not in stop:
            newTF.append(word)   

dataset2=[]
for i in range(len(sentences)):
    a=split_morphs(sentences[i])
    dataset= []
    for w in a:
        if w in newTF: 
            dataset.append(w)
    dataset2.append(dataset)
    
dataset3=[]
for k in range(len(dataset2)):
    if dataset2[k] != []:
        dataset3.append(dataset2[k])
 ############################ Apriori 알고리즘 적용 ################################
te = TransactionEncoder()
te_ary = te.fit(dataset3).transform(dataset3)
df = pd.DataFrame(te_ary, columns=te.columns_) #위에서 나온걸 보기 좋게 데이터프레임으로 변경
    
frequent_itemsets3 = apriori(df, min_support=0.003, max_len=3, use_colnames=True)
frequent_itemsets2 = apriori(df, min_support=0.003, max_len=2, use_colnames=True)

frequentThree=frequent_itemsets3[len(frequent_itemsets2):len(frequent_itemsets3)]
frequentThree=frequentThree.sort_values("support",ascending=False)

# 일정 확률 이상의 데이터 선택
frequentThree=frequentThree[frequentThree["support"]>=0.005]

# frozenset를 list로 변화
sets=[]
for line in frequentThree['itemsets']:
    sets.append(line)
association=[list(x) for x in sets]

In [13]:
############################### 나이브 베이지안 알고리즘 적용 ################################
spo = []
nonspo = []

for row in sentence_morphs:
    s1 = set(row)
    for dataset in association:
        s2 = set(dataset)
        isSpo = False
        if s2.issubset(s1) == True:
            spo.append(row)
            isSpo = True
            break
    if not isSpo:
        nonspo.append(row)
        
csv_file = open(ctitle, 'r')
doc = csv.reader(csv_file)
next(doc) # header 넘어가기

review_token = []
totalreview = []
for line in doc:
    text = line[2]
    totalreview.append(text) # totalreview 리스트에 결과 저장
    
index = 0
for comment in totalreview[0:1000]:
    review=preprocessing(comment) # 형태소 분석
    index+=1
    
    
    #__________prior probabilities
    prior_spo=len(spo)/(len(spo)+len(nonspo))
    prior_nonspo=len(nonspo)/(len(spo)+len(nonspo))
    
    #_________training data
    
    #spo의 중복단어 리스트 (the sum of term frequency for class SPO)
    spo_overlap=[]
    for i in range(len(spo)):
        spo_overlap.extend(spo[i])
        
    #nonspo의 중복단어 리스트 (the sum of term frequency for class NONSPO)
    nonspo_overlap=[]   
    for i in range(len(nonspo)):
        nonspo_overlap.extend(nonspo[i])
    
    #중복없는 전체단어 갯수: Vocabulary
    all_spo=[]
    for i in range(len(spo)):
        for j in range(len(spo[i])):
            if(spo[i][j] not in all_spo):
                all_spo.append(spo[i][j])
                
    all_nonspo=[]
    for i in range(len(nonspo)):
        for j in range(len(nonspo[i])):
            if(nonspo[i][j] not in all_nonspo):
                all_nonspo.append(nonspo[i][j])
    
    overall=[]
    overall.extend(all_spo)
    overall.extend(all_nonspo)
    
    nonover=[]
    for i in range(len(overall)):
        if(overall[i] not in nonover):
            nonover.append(overall[i])
    all_count=len(nonover)
    
    #_________likelihood
    
    #spo의  likelihood
    spo_like_result=1
    for i in range(len(review)):
        count=1# Laplace smoothing
        for j in range(len(spo_overlap)):
            if (review[i]==spo_overlap[j]):
                count=count+1
        spo_like_result=spo_like_result*(count/(len(spo_overlap)+all_count))#review 단어별 like
        
    
    #nonspo의  likelihood
    nonspo_like_result=1
    for i in range(len(review)):
        ncount=1# Laplace smoothing
        for j in range(len(nonspo_overlap)):
            if (review[i]==nonspo_overlap[j]):
                ncount=ncount+1
        nonspo_like_result=nonspo_like_result*(ncount/(len(nonspo_overlap)+all_count))#review 단어별 like 
        
    #_________posterior probabilities
    
    #P(SPO | REVIEW)=p(REVIEW|SPO)P(SPO)
    post_spo=spo_like_result*prior_spo
    
    #P(NONSPO | REVIEW)=P(REVIEW|NONSPO)P(NONSPO)
    post_nonspo=nonspo_like_result*prior_nonspo
    if(post_spo>post_nonspo):
        print(index,"번째 평점리뷰:",comment)
        print("분류결과: 스포 O")
        print("다음 review는 spo:{} >nonspo:{}이므로 스포이다.".format(post_spo,post_nonspo))
        print("=============================================================================================")
    
    elif(post_spo<post_nonspo):
        print(index,"번째 평점리뷰:",comment)
        print("분류결과: 스포 X")
        print("다음 review는 spo:{} <nonspo:{}이므로 스포가 아니다.".format(post_spo,post_nonspo))
        print("=============================================================================================")      
        

1 번째 평점리뷰: 관람객관객에게 미끼를 던진 영화  
분류결과: 스포 X
다음 review는 spo:5.4700580552314714e-17 <nonspo:1.6467326818433366e-14이므로 스포가 아니다.
2 번째 평점리뷰: 누가 황정민 연기 지겹다고 했냐. 이렇게 소름돋게 연기하는 배운데  
분류결과: 스포 X
다음 review는 spo:1.839471561441102e-30 <nonspo:5.1051995411029685e-30이므로 스포가 아니다.
3 번째 평점리뷰: 관람객반전에반전...감독한테놀아난느낌  
분류결과: 스포 X
다음 review는 spo:2.830624539902374e-26 <nonspo:5.624862763964073e-22이므로 스포가 아니다.
4 번째 평점리뷰: 관람객나감독.. 숨은 쉬게해줘야지...  
분류결과: 스포 X
다음 review는 spo:1.307843543964982e-15 <nonspo:8.255777159052684e-14이므로 스포가 아니다.
5 번째 평점리뷰: 정신적으로 너무 안좋다  
분류결과: 스포 X
다음 review는 spo:1.2960771371686381e-08 <nonspo:8.052134891003378e-08이므로 스포가 아니다.
6 번째 평점리뷰: 관람객...할말없는 영화임.본사람은 공감할듯..무섭고 잔인하고 징그럽고 소름끼치는 반전까지 고루 갖췄지만 그 모든걸 넘어서는 찝찝함.  
분류결과: 스포 X
다음 review는 spo:9.214961887559812e-57 <nonspo:9.85803838053712e-52이므로 스포가 아니다.
7 번째 평점리뷰: 관람객어이없어서 웃기고 그냥 보고난후 뭐지...?이느낌  
분류결과: 스포 X
다음 review는 spo:1.9352362582027448e-28 <nonspo:8.862566720497937e-27이므로 스포가 아니다.
8 번째 평점리뷰: 관람객보는 내내 골룸 생각났음ㅋㅋ  
분류결과: 스포 X
다음 review는 spo:

34 번째 평점리뷰: 솔직히 무슨재민지 하나도 모르겠다.. 시간날리고 온 기분  
분류결과: 스포 X
다음 review는 spo:1.665562548868557e-19 <nonspo:2.612968077331726e-18이므로 스포가 아니다.
35 번째 평점리뷰: 인류 역사상 가장 끔찍한 영화일 겁니다. 이런 걸 완성해 놓고  박수 치고 좋아했을 제작진을 생각하니 그게 더 오싹한 장면입니다. 대체 무슨 멘탈일까? 보는 내내 욕이 나오고 구역질이 나옵니다. 세기말적 영화입니다. 제발 보지 마시기를 강력 권유합니다  
분류결과: 스포 X
다음 review는 spo:7.00677208089586e-102 <nonspo:9.406856546220754e-96이므로 스포가 아니다.
36 번째 평점리뷰: 관람객잘만들었지만 관람은 비추입니다.  
분류결과: 스포 X
다음 review는 spo:7.166982620928102e-13 <nonspo:2.064297100752874e-11이므로 스포가 아니다.
37 번째 평점리뷰: 관람객성인배우는 물론 아역연기가..아주..후기 신경안쓰고 보고싶으면 보고 후회는 절대 안하실듯..  
분류결과: 스포 X
다음 review는 spo:1.8795690458480582e-45 <nonspo:5.77802538212287e-45이므로 스포가 아니다.
38 번째 평점리뷰: 내가 이걸 내돈주고 보게되다니...  
분류결과: 스포 X
다음 review는 spo:5.259671206953337e-20 <nonspo:3.215960710562125e-18이므로 스포가 아니다.
39 번째 평점리뷰: 최악의 좀비영화입니다  
분류결과: 스포 X
다음 review는 spo:5.375236965696078e-11 <nonspo:9.761831623115813e-10이므로 스포가 아니다.
40 번째 평점리뷰: 제발 이 영화 볼 돈으로 곱창사드세요....  
분류결과: 스포 X
다음 review는 spo:5.816933429499378e-23 <nonspo:

66 번째 평점리뷰: 알바천국이네 ㅋㅋㅋㅋ  
분류결과: 스포 X
다음 review는 spo:3.2401928429215953e-09 <nonspo:5.368089927335584e-09이므로 스포가 아니다.
67 번째 평점리뷰: 관람객몰입감 장난 아니네요 2시간 반이 길게 안느껴져요 다보고 나오는데 뭔가 지침ㅎㅎ 근데 좀 난해해서 한번 더 봐야겠음  
분류결과: 스포 X
다음 review는 spo:1.428511238925504e-51 <nonspo:2.17144597787469e-49이므로 스포가 아니다.
68 번째 평점리뷰: 관람객내용도 없고 잔인한 장면만 많네요 부성애를 자극하려면 결말이라도 잘 내던가하지 배우의 연기력은 최고 평점이지만 스토리는 정말 아닌듯 합니다. 보고 기분나쁜영화였습니다.  
분류결과: 스포 X
다음 review는 spo:3.6923973536808e-59 <nonspo:5.1609227602766934e-52이므로 스포가 아니다.
69 번째 평점리뷰: 비추 테러하라고 지령 떨어졌냐?정상적인 글에도 비추 10개씩은 기본이네  
분류결과: 스포 X
다음 review는 spo:2.7540297662011947e-35 <nonspo:2.3442242790778943e-33이므로 스포가 아니다.
70 번째 평점리뷰: 알바는 양심도 없나..내가 화가나서 평점쓰러 들어오긴 처음이다.어디서 되도안되는c급좀비물만들어와서 무슨 명감독인마냥 홍보하고 관객 낚이게 하는 영화.감독 반성 많이 하세요 인생 최악의영화중 한편으로 등극  
분류결과: 스포 X
다음 review는 spo:3.263253646978457e-107 <nonspo:5.561430926512005e-103이므로 스포가 아니다.
71 번째 평점리뷰: 관람객이상해요 무슨장르에요 이게  
분류결과: 스포 X
다음 review는 spo:2.6298356034766688e-20 <nonspo:1.567780846399036e-17이므로 스포가 아니다.
72 번째 평점리뷰: 평식이 믿고 지금 극

101 번째 평점리뷰: 관람객천우희에서 끝났어야되... 왜 갑자기 또 반전을 만들어서 에혀.. 좀비는 진짜 에러다..ㅡㅡ  
분류결과: 스포 X
다음 review는 spo:1.7401794124647747e-38 <nonspo:6.364837203431388e-36이므로 스포가 아니다.
102 번째 평점리뷰: 왜 곡성인줄 알거 같네 진짜 곡소리나네 아 내 시간  
분류결과: 스포 X
다음 review는 spo:3.406822162877749e-26 <nonspo:1.0295508894056708e-23이므로 스포가 아니다.
103 번째 평점리뷰: 0점 이없는게아쉽다...  
분류결과: 스포 X
다음 review는 spo:2.9297823685697063e-05 <nonspo:0.00018845216498904305이므로 스포가 아니다.
104 번째 평점리뷰: 관람객귀신이 예수 흉내내며 육체를 입기위해 사탄에게 사람을 제물로 바치는 인신제사를 모티브로 한 사타니즘 영화. 신을 희롱하고 무지한 관객들을 우롱한다. 영적인 시각이 없는 사람은 이 영화의 무서움을 알지못한 채 찝찝함만 남을 것. 역대 최'악'의 한국영화  
분류결과: 스포 X
다음 review는 spo:1.614833562668232e-119 <nonspo:7.38544165016942e-116이므로 스포가 아니다.
105 번째 평점리뷰: 처음에는 자극적이기도 하고 이 미스테리를 어떻게 풀까 기대하면서 봣는데 그냥 어이없게 끝나네  
분류결과: 스포 X
다음 review는 spo:2.4960761607363506e-39 <nonspo:1.6001425423998752e-38이므로 스포가 아니다.
106 번째 평점리뷰: 이따위를 쩝 시간아깝네  
분류결과: 스포 X
다음 review는 spo:3.2401928429215953e-09 <nonspo:1.0736179854671168e-08이므로 스포가 아니다.
107 번째 평점리뷰: 평점 0점이 없어서 1점줍니다.처음부터 끝까지 피토하고 죽이고..잔인하고 

132 번째 평점리뷰: 무당과.예수와.좀비와.악마와.귀신과.호러와.살인과...또 뭐가있뜨라...^#*!@?  
분류결과: 스포 X
다음 review는 spo:2.70176350786453e-33 <nonspo:7.088719825732498e-32이므로 스포가 아니다.
133 번째 평점리뷰: 관람객중간에나오고싶을정도..기분나쁜영화  
분류결과: 스포 X
다음 review는 spo:1.972376702607502e-18 <nonspo:1.3232673336241094e-15이므로 스포가 아니다.
134 번째 평점리뷰: 정말재미없다 보지마라  
분류결과: 스포 X
다음 review는 spo:2.1500947862784312e-11 <nonspo:2.614776327620307e-10이므로 스포가 아니다.
135 번째 평점리뷰: 관람객최고의 평론가가10점줘서 봤는데 평론가..급전필요했습니까??동생엄마애인같이봤는데 동생한테 미안했네요 이게 15세 말이 됩니까? 내 휴일 내 돈 너무 아까웠네요..볼건 배우들의 연기 연기는 정말 잘합니다.다만 영화가 정신병 걸리게 하는 더러운기분들게해요  
분류결과: 스포 X
다음 review는 spo:6.559206025201533e-105 <nonspo:2.8645382529938106e-100이므로 스포가 아니다.
136 번째 평점리뷰: 전문가 평점 높아서 봤는데,, 그닼.. 추격자 때 포스는 안나오네요, 긴장감이나 몰입도는 있는데, 잔인하고 징그러운 장면이 많습니다. 개인적으로 좀비씬은 그냥 없었다면 영화의 퀄리티가 더 높아지지 않았을까 싶습니다. 전문가 중에도 알바가 있는듯..ㅜㅜ  
분류결과: 스포 X
다음 review는 spo:2.8956404151725744e-84 <nonspo:1.2720062113944542e-78이므로 스포가 아니다.
137 번째 평점리뷰:  잘 모르겠다는..이게 그렇게 칭찬받을 스토리인가?내가 이해력이 딸리는건지...하여간 뭐...괜찮긴한데 아주 내인생 영화는 아니고. 사실 이해가 잘 안됩니다. 배

...

868 번째 평점리뷰: 개봉전 평점에 현혹되지 말라!! 오랜만에 영화보고 돈 아까웠다 거의 3시간동안 집중해서 봤다 하지만 도대체 감독은 뭘 말하고 싶은지.대충 내용과 의도를 기사를 통해봐서 알았지만.이건 아닌듯.재미도 없고 의미도 없으며 징그럽고잔인..19금삼류좀비짬뽕영화  
분류결과: 스포 X
다음 review는 spo:1.4674817380885792e-109 <nonspo:1.181801852287504e-100이므로 스포가 아니다.
869 번째 평점리뷰: 진짜 평점 보고 말이 안나온다 이게 개연성이 없다고 느끼면 이해를 못하는거라고??? 진짜 이거 평점 높게 준 사람 대부분은 자신은 남과는 다른 비범한 사람이고 지 잘난 맛에 사는 인간들 대부분 정말 역겹다 절대 보지말라고 하고싶다 이런 기분더러운영화  
분류결과: 스포 X
다음 review는 spo:1.8141131630220627e-93 <nonspo:9.949221070821406e-87이므로 스포가 아니다.
870 번째 평점리뷰: 관람객최악입니다 감독님팬이라 부모님 모시고 갔는대  
분류결과: 스포 X
다음 review는 spo:5.332188977041097e-24 <nonspo:1.3397546115185743e-22이므로 스포가 아니다.
871 번째 평점리뷰: 아니.... 솔직하자.  평점에 솔직하자. 안본 사람 낚는 것도 아니고 우리만 낚이자  
분류결과: 스포 X
다음 review는 spo:9.694889049165631e-25 <nonspo:1.717634117331506e-24이므로 스포가 아니다.
872 번째 평점리뷰: 한국영화가 할 수있다를 보여준 영화가 아닌한국영화는 여기가 끝이다를 보여준 영화.그 끝을 어떻게든 포장하는 평론가들  
분류결과: 스포 X
다음 review는 spo:4.982701881228196e-46 <nonspo:3.245498497397968e-40이므로 스포가 아니다.
873 번째 평점리뷰: 이동진평론가님 평론보고 갔는데 제스타일은 아니네요. 개연성0 남

899 번째 평점리뷰: 주머니속 만원이 꼴보기 싫을때 보는영화  
분류결과: 스포 X
다음 review는 spo:1.3149178017383344e-19 <nonspo:3.5643564542063546e-18이므로 스포가 아니다.
900 번째 평점리뷰: 기본적인 개연성을 상실한 영화이다. 무수히 많은 말도안되는 설정 뻔히 보이는 설정들 .,,뭔가 있어보일려고 만든 영화.뭔가 있어보일려고 하는 감독. 정신차리길 바랍니다.제발.  
분류결과: 스포 X
다음 review는 spo:4.4170423334404525e-74 <nonspo:1.399969633858343e-70이므로 스포가 아니다.
901 번째 평점리뷰: 여지껏 봤던 영화중 가장별로돈아깝다고 얘기하는애들 우습고한심했는데 영화보고집에돌아오면서 이해가되더라  
분류결과: 스포 X
다음 review는 spo:1.1717112007591284e-50 <nonspo:8.361514645469472e-46이므로 스포가 아니다.
902 번째 평점리뷰: 모든의미에서 쓰레기다  
분류결과: 스포 X
다음 review는 spo:2.2934344386969927e-11 <nonspo:1.174355683983857e-10이므로 스포가 아니다.
903 번째 평점리뷰: 이 영화  뭐냐 해석들은 잘하네  
분류결과: 스포 X
다음 review는 spo:9.511589410654418e-14 <nonspo:2.1772671796558083e-11이므로 스포가 아니다.
904 번째 평점리뷰: 배우들의 연기력으로 덮어둔 엉성한 시나리오에 놀랐습니다. 작품에 대한 다양한 해석이 나오는 건 고무적이나, 거창하게 종교철학적 담론을 담은 척하는 '포장'에 화가 나네요. 그저 킬링타임하기에도 아까운, 자극적인 필름인데 아니라고 주장하는 좀비호러물.  
분류결과: 스포 X
다음 review는 spo:6.462336015747332e-119 <nonspo:8.104141469447091e-113이므로 스포가 아니다.
905 번째 평점리뷰: 관람객

930 번째 평점리뷰: 반전은 무슨 반전?? 배우들 연기력 빼고는 스토리에 연출이 최악이구만... 꼴에 놀란의 열린 결말을 따라하는건가?ㅋㅋㅋ 스크린쿼터 아니였으면 100만도 못했을영화. 특히! 그놈의 한국영화들 사투리좀 그만 쳐넣어라 먼말인지 알아들어 쳐먹어야 영화를보지  
분류결과: 스포 X
다음 review는 spo:4.614702127040243e-109 <nonspo:4.926900757777696e-100이므로 스포가 아니다.
931 번째 평점리뷰: 오컬트고 맥거핀이고 간에, "벌거벗은임금님"이 떠오르는 영화. 평론과 해석은 화려하나, 평범한 관객에겐 끔찍한 징그러움 외엔 아무느낌 없다. 스릴러라는 장르에 낚였다. 그냥 좀비호러물이다.  
분류결과: 스포 X
다음 review는 spo:1.8802004141577748e-100 <nonspo:3.6667856268925506e-94이므로 스포가 아니다.
932 번째 평점리뷰: 관람객웰메이드 쓰레기, 영화같은 똥  
분류결과: 스포 X
다음 review는 spo:9.649856386030819e-27 <nonspo:1.9521905452494471e-25이므로 스포가 아니다.
933 번째 평점리뷰: 관람객찝찝하고 너무 짜내어 억지로 만들어낸 반전들 같은 느낌....  
분류결과: 스포 X
다음 review는 spo:1.8972904492183777e-31 <nonspo:3.262502270156128e-27이므로 스포가 아니다.
934 번째 평점리뷰: 0점은 없냐? 10자10자  
분류결과: 스포 X
다음 review는 spo:3.2401928429215953e-09 <nonspo:7.730049495363242e-07이므로 스포가 아니다.
935 번째 평점리뷰: 사람의 영혼을 파괴하는 질낮은 영화  
분류결과: 스포 X
다음 review는 spo:5.259671206953338e-19 <nonspo:8.910891135515887e-18이므로 스포가 아니다.
936 번째 평점리뷰: 대체 왜 이런 영화

963 번째 평점리뷰: 와나 ㅋ 평생 처음 평점주네여; 이끼같은 영화라 생각하고 잔인하거나 무서운 장면 감안하고 봤지만. 이거 무슨 삼류 포르노같은 기분이예요 자극적인것만 첨부터 끝까지 다 때려놓고 돈벌려고 작정한 영화 같아요 배우들 연기력이 안타까움  
분류결과: 스포 X
다음 review는 spo:6.902968050067776e-100 <nonspo:6.379161322972032e-94이므로 스포가 아니다.
964 번째 평점리뷰:  꼭 어렵게, 현학적 수사를 덧붙이고 꼬아서 써야만 좋은 글인가? 혹자는 좋은 글이야 말로 독자들에게 쉽게 읽히면서도 깊은 의미를 부여하는 글이라 말했다. 곡성은 관객들에게 쉽게 읽히지도 않을뿐더러, 무엇을 말하고자 하는지마저 베베 꼬아놨다.  
분류결과: 스포 X
다음 review는 spo:7.701300896390246e-102 <nonspo:2.2125366736618078e-97이므로 스포가 아니다.
965 번째 평점리뷰: 기분나쁘고 찝찝한걸 제하고 보더라도, 도무지 스토리가 말이안된다. 반전에 집착하다 개연성을 잃은 전형적인 폐급 시나리오고  아역배우도 걱정됨. 끝으로 이 영화 종니 빨아재끼던 언론과 평론가 쉐이들 니네는 형한테 싸다뒤 한대씩 맞아야해. 아오!  
분류결과: 스포 X
다음 review는 spo:5.0876948460559567e-129 <nonspo:3.00194404930405e-123이므로 스포가 아니다.
966 번째 평점리뷰: 진짜 스트레스 받는 영화다 돈아까움  
분류결과: 스포 X
다음 review는 spo:1.3149178017383348e-18 <nonspo:6.41584161757144e-16이므로 스포가 아니다.
967 번째 평점리뷰: 평점 처음 써봅니다 배우들의 연기는 좋았으나 잔인하고 재미도 없었습니다  
분류결과: 스포 X
다음 review는 spo:1.2879341656555802e-24 <nonspo:6.611761136147471e-22이므로 스포가 아니다.
968 번째 평점

993 번째 평점리뷰: 영화보면선 이해안가고 뒤늦게 본 해석으로도 억지라는 생각을 지울수가 없음  
분류결과: 스포 X
다음 review는 spo:4.824928193015412e-25 <nonspo:6.793623097468076e-22이므로 스포가 아니다.
994 번째 평점리뷰: 솔직히 1점도 아깝다....처음부터 끝까지 난해하기만하고 내용도 너무 꼬아놓았고 계속 소리지르고 굿씬까지있어서 이건 뭐 그냥 소음공해수준;;  
분류결과: 스포 X
다음 review는 spo:3.3578348950470234e-61 <nonspo:3.799756689793739e-59이므로 스포가 아니다.
995 번째 평점리뷰: <스포X> 돈주고 보지 마세요 20개이상의 복선과 단서를 뿌려놓고 회수하는건 3개도 안됩니다 그냥 다른영화 보세요 2시간반짜린데 알맹이는 1시간도 안됩니다 시간아깝고 보며서 지루하고 졸립니다 보면 후회하실 거에요 지인들한테만 물어봐도 바로 알수 있어요  
분류결과: 스포 X
다음 review는 spo:3.948349430142714e-89 <nonspo:7.601104558273489e-86이므로 스포가 아니다.
996 번째 평점리뷰: 관람객하...할말없는영화  
분류결과: 스포 X
다음 review는 spo:2.150094786278431e-11 <nonspo:1.830343429334215e-09이므로 스포가 아니다.
997 번째 평점리뷰: 관람객곡성 - 후까시 = 0본인이 뭘 만들고 있는지 모를 때 나오는 결말  
분류결과: 스포 X
다음 review는 spo:2.5158071356635686e-27 <nonspo:4.4202051518483456e-24이므로 스포가 아니다.
998 번째 평점리뷰: 두시간 사십분동안  떡밥만 던지고 황정민이 악마랑한패니  알아서 해석해라  드럽게 무책임한  찜찜한 결말 스포? 알아도 전혀 상관없음  하도 스포로 이슈되서 극장가서본건데 이게 낚인거임 ㅋㅋ  
분류결과: 스포 X
다음 review는 spo:1.486006